In [1]:
from openai import OpenAI
import os

# API 키 직접 넣거나 (연습용)
#client = OpenAI(api_key="")
#client

In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client

In [4]:
import os, json
from typing import Dict, Any

from openai import OpenAI

# =========================
# 설정
# =========================
MODEL = "gpt-4.1-mini"  # 너희 계정에서 되는 모델로 바꿔도 됨
DRIVERS = [
    "Trust",
    "Communication",
    "Decision Making",
    "Innovative Thinking",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]

# JSON Schema (AI2 출력 강제)
AI2_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "project_context": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "urgency": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "ambiguity": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "external_collab": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "conflict_risk": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "innovation_need": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            },
            "required": ["urgency", "ambiguity", "external_collab", "conflict_risk", "innovation_need"],
        },
        "constraints": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "team_size": {"type": "integer", "minimum": 2, "maximum": 10},
                "must_include_dept": {"type": "array", "items": {"type": "string"}, "maxItems": 2},
                "max_moves_2y_avg": {"type": "number", "minimum": 0.0, "maximum": 10.0},
            },
            "required": ["team_size", "must_include_dept", "max_moves_2y_avg"],
        },
        "ai2_initial_weights": {
            "type": "object",
            "additionalProperties": False,
            "properties": {d: {"type": "number", "minimum": 0.0, "maximum": 1.0} for d in DRIVERS},
            "required": DRIVERS,
        },
        "rationale": {"type": "string"},
        "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
    },
    "required": ["project_context", "constraints", "ai2_initial_weights", "rationale", "confidence"],
}

def _normalize_weights(w: Dict[str, float], eps: float = 1e-6) -> Dict[str, float]:
    w2 = {d: max(float(w.get(d, 0.0)), eps) for d in DRIVERS}
    s = sum(w2.values())
    return {d: w2[d] / s for d in DRIVERS}

def _clamp01(x: float) -> float:
    return max(0.0, min(1.0, float(x)))

def _validate_ai2(out: Dict[str, Any]) -> Dict[str, Any]:
    # (1) context clamp
    ctx = out["project_context"]
    for k in ctx:
        ctx[k] = _clamp01(ctx[k])

    # (2) constraints safety
    c = out["constraints"]
    c["team_size"] = int(max(2, min(10, c["team_size"])))
    c["max_moves_2y_avg"] = float(max(0.0, min(10.0, c["max_moves_2y_avg"])))
    if len(c.get("must_include_dept", [])) > 2:
        c["must_include_dept"] = c["must_include_dept"][:2]

    # (3) weights normalize
    out["ai2_initial_weights"] = _normalize_weights(out["ai2_initial_weights"])

    # (4) confidence clamp
    out["confidence"] = _clamp01(out.get("confidence", 0.6))

    return out

def ai2_context_parse(project_prompt: str) -> Dict[str, Any]:
    """
    프로젝트 텍스트 -> (컨텍스트 0~1) + (제약조건) + (7드라이버 초기 가중치) + (근거/신뢰도)
    """
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다. os.environ['OPENAI_API_KEY']='sk-...' 로 세팅하세요.")

    client = OpenAI(api_key=api_key)

    system = """
너는 '프로젝트 요구사항 텍스트'를 읽고 팀 매칭에 필요한 컨텍스트를 구조화하는 분석기다.
반드시 JSON 스키마에 맞는 JSON만 출력한다.

스케일 가이드(0~1):
- urgency(긴급도): 0=여유, 1=매우 촉박(예: 1~2주 마감)
- ambiguity(불확실성): 0=요구사항 명확, 1=자주 바뀌거나 처음 하는 일
- external_collab(대외/타부서 협업): 0=내부 소수, 1=이해관계자/부서가 많음
- conflict_risk(갈등 위험): 0=낮음, 1=의견 충돌/조정 필요 큼
- innovation_need(혁신 필요): 0=정해진 방식, 1=새로운 시도/실험 필수

제약조건은 너무 빡세지 않게(데모에서도 항상 팀이 뽑히도록) 제시해라.
근거가 약하면 0.5 근처로 두고 confidence를 낮춰라.
""".strip()

    user = f"""
[프로젝트 요구사항]
{project_prompt}

[출력 요구]
- project_context: 위 5개 항목을 0~1로
- constraints: team_size(2~10), must_include_dept(0~2개), max_moves_2y_avg(0~10)
- ai2_initial_weights: 7드라이버 중요도(0~1) (합은 나중에 정규화)
- rationale: 한국어 3~6문장
- confidence: 0~1
""".strip()

    try:
        resp = client.responses.create(
            model=MODEL,
            input=[
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
            text={
                "format": {
                    "type": "json_schema",
                    "name": "AI2Context",
                    "schema": AI2_SCHEMA,
                    "strict": True,
                }
            },
        )
        out = json.loads(resp.output_text)
        out = _validate_ai2(out)
        return out

    except Exception as e:
        # ✅ 안전한 fallback (발표/데모용)
        fallback = {
            "project_context": {
                "urgency": 0.5,
                "ambiguity": 0.5,
                "external_collab": 0.5,
                "conflict_risk": 0.5,
                "innovation_need": 0.5,
            },
            "constraints": {
                "team_size": 4,
                "must_include_dept": [],
                "max_moves_2y_avg": 3.0,
            },
            "ai2_initial_weights": _normalize_weights({d: 1.0 for d in DRIVERS}),
            "rationale": f"AI2 호출 오류로 중립값(fallback)을 사용했습니다. 오류: {type(e).__name__}",
            "confidence": 0.2,
        }
        return fallback


In [5]:
project_prompt = """
대외 협업이 많고 마감이 빠른 프로젝트입니다.
요구사항이 자주 바뀌고 이해관계자가 많아 조율이 필요합니다.
핵심은 빠른 의사결정과 커뮤니케이션이고, 갈등이 발생할 수 있습니다.
"""

result = ai2_context_parse(project_prompt)

print(json.dumps(result, ensure_ascii=False, indent=2))
print("\n가중치 합:", sum(result["ai2_initial_weights"].values()))


{
  "project_context": {
    "urgency": 0.9,
    "ambiguity": 0.8,
    "external_collab": 0.9,
    "conflict_risk": 0.7,
    "innovation_need": 0.5
  },
  "constraints": {
    "team_size": 5,
    "must_include_dept": [
      "외부 협업 담당"
    ],
    "max_moves_2y_avg": 5.0
  },
  "ai2_initial_weights": {
    "Trust": 0.13207547169811323,
    "Communication": 0.16981132075471703,
    "Decision Making": 0.17924528301886794,
    "Innovative Thinking": 0.09433962264150945,
    "Psychological Safety": 0.15094339622641514,
    "Conflict Management": 0.16037735849056606,
    "Role Definition": 0.11320754716981134
  },
  "rationale": "프로젝트 요구사항에 따르면 대외 협업이 많고 마감이 빠르며, 요구사항 변경이 빈번하고 이해관계자 조율이 필요합니다. 따라서 긴급도(urgency)와 대외 협업(external_collab), 불확실성(ambiguity)이 매우 높게 평가되었습니다. 갈등 가능성이 있어 conflict_risk도 높게 설정했습니다. 핵심 드라이버는 빠른 의사결정과 커뮤니케이션으로, 이에 해당 요소들의 가중치를 높였습니다. 혁신의 필요성은 명확하게 언급되지 않아 중간 수준으로 두었고 팀 크기는 적정한 5명으로 제안합니다.",
  "confidence": 0.85
}

가중치 합: 1.0000000000000002


In [6]:
"""
3가지 프레임워크(Framework1/2/3)에서 나온 결과를
ML2(통합 ML 파이프라인 입력 포맷)로 “변환”만 하는 부분.

- 입력: framework별 산출물(list[dict] 또는 dict)
- 출력: ML2 표준 스키마(list[dict])
"""

from __future__ import annotations
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Optional


# ---------------------------
# 1) ML2 표준 스키마 정의
# ---------------------------
@dataclass
class ML2Item:
    # 공통 식별
    use_case_id: str
    name: str

    # ML2에서 바로 쓰기 좋게: 문제정의/입력/출력/라벨/평가지표/리스크
    problem_type: str                # "classification" | "regression" | "ranking" | "generation" | "anomaly" ...
    input_sources: List[str]         # 예: ["HRIS", "LMS", "Slack metadata"]
    target_label: Optional[str]      # 지도학습이면 라벨, 아니면 None
    prediction_output: str           # 예: "team_fit_score", "risk_flag"

    metrics: List[str]               # 예: ["F1", "AUROC", "Calibration"]
    constraints: List[str]           # 예: ["PII_minimization", "Fairness", "Explainability"]
    notes: Optional[str] = None

    # 추적용
    origin_framework: str = ""       # "F1" | "F2" | "F3"
    origin_ref: Optional[str] = None # 원본 항목 id 등


# ---------------------------
# 2) 프레임워크별 -> ML2 변환기
#    (여기만 수정해서 너희 포맷에 맞추면 됨)
# ---------------------------
def map_framework1_to_ml2(items: List[Dict[str, Any]]) -> List[ML2Item]:
    out: List[ML2Item] = []
    for it in items:
        out.append(
            ML2Item(
                use_case_id=str(it.get("id", it.get("use_case_id", "F1-UNKNOWN"))),
                name=str(it.get("title", it.get("name", "Untitled"))),
                problem_type=str(it.get("ml_task", "classification")),
                input_sources=list(it.get("data_sources", [])),
                target_label=it.get("label"),
                prediction_output=str(it.get("output", "prediction")),
                metrics=list(it.get("metrics", ["F1"])),
                constraints=list(it.get("constraints", [])),
                notes=it.get("notes"),
                origin_framework="F1",
                origin_ref=str(it.get("id")) if it.get("id") is not None else None,
            )
        )
    return out


def map_framework2_to_ml2(items: List[Dict[str, Any]]) -> List[ML2Item]:
    out: List[ML2Item] = []
    for it in items:
        # 예: framework2가 "inputs"/"kpi"/"risk" 같은 키를 쓴다고 가정
        out.append(
            ML2Item(
                use_case_id=str(it.get("key", "F2-UNKNOWN")),
                name=str(it.get("usecase", it.get("name", "Untitled"))),
                problem_type=str(it.get("type", "classification")),
                input_sources=list(it.get("inputs", [])),
                target_label=it.get("target"),
                prediction_output=str(it.get("predict", "prediction")),
                metrics=list(it.get("kpi", ["F1"])),
                constraints=list(it.get("risk", [])),
                notes=it.get("memo"),
                origin_framework="F2",
                origin_ref=str(it.get("key")) if it.get("key") is not None else None,
            )
        )
    return out


def map_framework3_to_ml2(items: List[Dict[str, Any]]) -> List[ML2Item]:
    out: List[ML2Item] = []
    for it in items:
        out.append(
            ML2Item(
                use_case_id=str(it.get("uid", "F3-UNKNOWN")),
                name=str(it.get("scenario", it.get("name", "Untitled"))),
                problem_type=str(it.get("task", "classification")),
                input_sources=list(it.get("source_systems", [])),
                target_label=it.get("y_label"),
                prediction_output=str(it.get("y_hat", "prediction")),
                metrics=list(it.get("eval", ["F1"])),
                constraints=list(it.get("governance", [])),
                notes=it.get("comment"),
                origin_framework="F3",
                origin_ref=str(it.get("uid")) if it.get("uid") is not None else None,
            )
        )
    return out


# ---------------------------
# 3) 3개 프레임워크 결과를 ML2로 합치기(중복 제거 옵션)
# ---------------------------
def to_ml2(
    f1: List[Dict[str, Any]],
    f2: List[Dict[str, Any]],
    f3: List[Dict[str, Any]],
    dedupe_by: str = "use_case_id",  # or "name"
) -> List[Dict[str, Any]]:
    ml2_items: List[ML2Item] = []
    ml2_items += map_framework1_to_ml2(f1)
    ml2_items += map_framework2_to_ml2(f2)
    ml2_items += map_framework3_to_ml2(f3)

    seen = set()
    deduped: List[ML2Item] = []
    for it in ml2_items:
        key = getattr(it, dedupe_by)
        if key in seen:
            continue
        seen.add(key)
        deduped.append(it)

    # 최종: ML2 표준 dict 리스트로 반환 (JSON 저장/DF화 편함)
    return [asdict(x) for x in deduped]


# ---------------------------
# 4) 사용 예시
# ---------------------------
if __name__ == "__main__":
    framework1_results = [
        {"id": "UC-001", "title": "팀 조합 추천", "ml_task": "ranking", "data_sources": ["HRIS", "Survey"], "label": None,
         "output": "team_rank", "metrics": ["NDCG", "HitRate"], "constraints": ["Fairness", "PII_minimization"]}
    ]
    framework2_results = [
        {"key": "UC-002", "usecase": "이직 위험 예측", "type": "classification", "inputs": ["HRIS", "Attendance"],
         "target": "attrition_90d", "predict": "attrition_prob", "kpi": ["AUROC", "PR-AUC"], "risk": ["Explainability"]}
    ]
    framework3_results = [
        {"uid": "UC-003", "scenario": "성과 하락 탐지", "task": "anomaly", "source_systems": ["PerformanceReview"],
         "y_label": None, "y_hat": "drop_flag", "eval": ["Precision@K"], "governance": ["Audit_log"]}
    ]

    ml2 = to_ml2(framework1_results, framework2_results, framework3_results, dedupe_by="use_case_id")
    for row in ml2:
        print(row)


{'use_case_id': 'UC-001', 'name': '팀 조합 추천', 'problem_type': 'ranking', 'input_sources': ['HRIS', 'Survey'], 'target_label': None, 'prediction_output': 'team_rank', 'metrics': ['NDCG', 'HitRate'], 'constraints': ['Fairness', 'PII_minimization'], 'notes': None, 'origin_framework': 'F1', 'origin_ref': 'UC-001'}
{'use_case_id': 'UC-002', 'name': '이직 위험 예측', 'problem_type': 'classification', 'input_sources': ['HRIS', 'Attendance'], 'target_label': 'attrition_90d', 'prediction_output': 'attrition_prob', 'metrics': ['AUROC', 'PR-AUC'], 'constraints': ['Explainability'], 'notes': None, 'origin_framework': 'F2', 'origin_ref': 'UC-002'}
{'use_case_id': 'UC-003', 'name': '성과 하락 탐지', 'problem_type': 'anomaly', 'input_sources': ['PerformanceReview'], 'target_label': None, 'prediction_output': 'drop_flag', 'metrics': ['Precision@K'], 'constraints': ['Audit_log'], 'notes': None, 'origin_framework': 'F3', 'origin_ref': 'UC-003'}


In [13]:
import os, json
from typing import Dict, Any, Tuple

from openai import OpenAI

# =========================
# 설정
# =========================
MODEL = "gpt-4.1-mini"  # 너희 계정에서 되는 모델로 바꿔도 됨
DRIVERS = [
    "Trust",
    "Communication",
    "Decision Making",
    "Innovative Thinking",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]

# -------------------------
# AI2 JSON Schema (출력 강제)
# -------------------------
AI2_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "project_context": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "urgency": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "ambiguity": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "external_collab": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "conflict_risk": {"type": "number", "minimum": 0.0, "maximum": 1.0},
                "innovation_need": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            },
            "required": ["urgency", "ambiguity", "external_collab", "conflict_risk", "innovation_need"],
        },
        "constraints": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "team_size": {"type": "integer", "minimum": 2, "maximum": 10},
                "must_include_dept": {"type": "array", "items": {"type": "string"}, "maxItems": 2},
                "max_moves_2y_avg": {"type": "number", "minimum": 0.0, "maximum": 10.0},
            },
            "required": ["team_size", "must_include_dept", "max_moves_2y_avg"],
        },
        "ai2_initial_weights": {
            "type": "object",
            "additionalProperties": False,
            "properties": {d: {"type": "number", "minimum": 0.0, "maximum": 1.0} for d in DRIVERS},
            "required": DRIVERS,
        },
        "rationale": {"type": "string"},
        "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
    },
    "required": ["project_context", "constraints", "ai2_initial_weights", "rationale", "confidence"],
}

# =========================
# 유틸
# =========================
def _clamp01(x: float) -> float:
    return max(0.0, min(1.0, float(x)))

def normalize_weights(w: Dict[str, float], eps: float = 1e-6) -> Dict[str, float]:
    w2 = {d: max(float(w.get(d, 0.0)), eps) for d in DRIVERS}
    s = sum(w2.values())
    return {d: w2[d] / s for d in DRIVERS}

def validate_ai2(out: Dict[str, Any]) -> Dict[str, Any]:
    # 1) context clamp
    ctx = out["project_context"]
    for k in ctx:
        ctx[k] = _clamp01(ctx[k])

    # 2) constraints safety
    c = out["constraints"]
    c["team_size"] = int(max(2, min(10, c["team_size"])))
    c["max_moves_2y_avg"] = float(max(0.0, min(10.0, c["max_moves_2y_avg"])))
    if len(c.get("must_include_dept", [])) > 2:
        c["must_include_dept"] = c["must_include_dept"][:2]

    # 3) weights normalize
    out["ai2_initial_weights"] = normalize_weights(out["ai2_initial_weights"])
    out["confidence"] = _clamp01(out.get("confidence", 0.6))
    return out

# =========================
# (1) AI2: Prompt -> Context JSON
# =========================
def ai2_context_parse(project_prompt: str) -> Dict[str, Any]:
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다.")

    client = OpenAI(api_key=api_key)

    system = """
너는 '프로젝트 요구사항 텍스트'를 읽고 팀 매칭에 필요한 컨텍스트를 구조화하는 분석기다.
반드시 JSON 스키마에 맞는 JSON만 출력한다.

스케일 가이드(0~1):
- urgency(긴급도): 0=여유, 1=매우 촉박(예: 1~2주 마감)
- ambiguity(불확실성): 0=요구사항 명확, 1=자주 바뀌거나 처음 하는 일
- external_collab(대외/타부서 협업): 0=내부 소수, 1=이해관계자/부서가 많음
- conflict_risk(갈등 위험): 0=낮음, 1=의견 충돌/조정 필요 큼
- innovation_need(혁신 필요): 0=정해진 방식, 1=새로운 시도/실험 필수

제약조건은 너무 빡세지 않게(데모에서도 항상 팀이 뽑히도록) 제시해라.
근거가 약하면 0.5 근처로 두고 confidence를 낮춰라.
""".strip()

    user = f"""
[프로젝트 요구사항]
{project_prompt}

[출력 요구]
- project_context: 위 5개 항목을 0~1로
- constraints: team_size(2~10), must_include_dept(0~2개), max_moves_2y_avg(0~10)
- ai2_initial_weights: 7드라이버 중요도(0~1) (합은 나중에 정규화)
- rationale: 한국어 3~6문장
- confidence: 0~1
""".strip()

    try:
        resp = client.responses.create(
            model=MODEL,
            input=[
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
            text={
                "format": {
                    "type": "json_schema",
                    "name": "AI2Context",
                    "schema": AI2_SCHEMA,
                    "strict": True,
                }
            },
        )
        out = json.loads(resp.output_text)
        return validate_ai2(out)

    except Exception as e:
        # fallback: 중립값
        fallback = {
            "project_context": {
                "urgency": 0.5, "ambiguity": 0.5, "external_collab": 0.5,
                "conflict_risk": 0.5, "innovation_need": 0.5
            },
            "constraints": {"team_size": 4, "must_include_dept": [], "max_moves_2y_avg": 3.0},
            "ai2_initial_weights": normalize_weights({d: 1.0 for d in DRIVERS}),
            "rationale": f"AI2 호출 오류로 중립값(fallback)을 사용했습니다. 오류: {type(e).__name__}",
            "confidence": 0.2,
        }
        return fallback

# =========================
# (2) 3 Frameworks: Context -> Prior Weights
# =========================
def framework_A_mckinsey_prior(ctx: Dict[str, float]) -> Tuple[Dict[str,float], str]:
    """
    프레임워크 A: '프로젝트 상황을 McKinsey식 드라이버 중요도로 매핑' (도메인 룰)
    """
    w = {d: 1.0 for d in DRIVERS}

    # 대외협업↑ => 소통↑
    w["Communication"] += 2.0 * ctx["external_collab"]

    # 마감 촉박↑ => 의사결정↑ + 역할명확성↑
    w["Decision Making"] += 1.6 * ctx["urgency"]
    w["Role Definition"] += 1.2 * ctx["urgency"]

    # 혁신 필요↑ => 혁신적 사고↑
    w["Innovative Thinking"] += 1.4 * ctx["innovation_need"]

    # 불확실성↑ => 신뢰/심리적안전감↑ (혼돈에서 버티기)
    w["Trust"] += 0.8 * ctx["ambiguity"]
    w["Psychological Safety"] += 1.0 * ctx["ambiguity"]

    return normalize_weights(w), "대외협업/마감/혁신/불확실성 신호를 드라이버 중요도로 룰 매핑"

def framework_B_risk_prior(ctx: Dict[str, float]) -> Tuple[Dict[str,float], str]:
    """
    프레임워크 B: '리스크 관점' (갈등/불확실성이 크면 안전장치 드라이버를 올림)
    """
    w = {d: 1.0 for d in DRIVERS}

    # 갈등 위험↑ => 갈등관리↑ + 심리적 안전감↑
    w["Conflict Management"] += 2.2 * ctx["conflict_risk"]
    w["Psychological Safety"] += 1.5 * ctx["conflict_risk"]

    # 불확실성↑ => 신뢰↑ + 소통↑ (정보정렬 필요)
    w["Trust"] += 1.2 * ctx["ambiguity"]
    w["Communication"] += 0.9 * ctx["ambiguity"]

    return normalize_weights(w), "갈등/불확실성 중심으로 리스크 완화 드라이버 비중 강화"

def framework_C_execution_prior(ctx: Dict[str, float]) -> Tuple[Dict[str,float], str]:
    """
    프레임워크 C: '실행(Delivery) 관점' (마감/협업은 운영 난이도를 올림)
    """
    w = {d: 1.0 for d in DRIVERS}

    # 마감 촉박↑ => 의사결정↑ + 역할명확성↑
    w["Decision Making"] += 1.8 * ctx["urgency"]
    w["Role Definition"] += 1.6 * ctx["urgency"]

    # 대외협업↑ => 소통↑ + 신뢰↑(핸드오프 많음)
    w["Communication"] += 1.4 * ctx["external_collab"]
    w["Trust"] += 0.7 * ctx["external_collab"]

    return normalize_weights(w), "마감/핸드오프/운영 관점에서 의사결정·역할·소통을 강화"

# =========================
# (3) 앙상블 Prior: AI2 + A/B/C 결합
# =========================
def ensemble_prior(
    ai2_w: Dict[str,float],
    wA: Dict[str,float],
    wB: Dict[str,float],
    wC: Dict[str,float],
    alpha=(0.40, 0.20, 0.20, 0.20)
) -> Dict[str,float]:
    a1,a2,a3,a4 = alpha
    w = {d: a1*ai2_w[d] + a2*wA[d] + a3*wB[d] + a4*wC[d] for d in DRIVERS}
    return normalize_weights(w)

# =========================
# (4) Step2-2 Orchestrator
# =========================
def step2_2_ai2_plus_frameworks(project_prompt: str, alpha=(0.40,0.20,0.20,0.20)) -> Dict[str,Any]:
    ai2_out = ai2_context_parse(project_prompt)
    ctx = ai2_out["project_context"]

    wA, whyA = framework_A_mckinsey_prior(ctx)
    wB, whyB = framework_B_risk_prior(ctx)
    wC, whyC = framework_C_execution_prior(ctx)

    prior = ensemble_prior(ai2_out["ai2_initial_weights"], wA, wB, wC, alpha=alpha)

    return {
        "ai2_output": ai2_out,
        "framework_weights": {"A": wA, "B": wB, "C": wC},
        "ensemble_prior": prior,
        "explain": {
            "alpha": {"AI2": alpha[0], "A": alpha[1], "B": alpha[2], "C": alpha[3]},
            "why_framework_A": whyA,
            "why_framework_B": whyB,
            "why_framework_C": whyC,
            "note": "ensemble_prior는 ML2(가중치 최적화)로 들어가는 prior(초기값)입니다."
        }
    }


In [14]:
project_prompt = """
대외 협업이 많고 마감이 빠른 프로젝트입니다. 팀 사이즈는 4명으로 하려고 합니다.
요구사항이 자주 바뀌고 이해관계자가 많아 조율이 필요합니다.
핵심은 빠른 의사결정과 커뮤니케이션이고, 갈등이 발생할 수 있습니다.
"""

out = step2_2_ai2_plus_frameworks(project_prompt, alpha=(0.40,0.20,0.20,0.20))

print("=== AI2 Output (Context/Constraints/AI2 Weights) ===")
print(json.dumps(out["ai2_output"], ensure_ascii=False, indent=2))

print("\n=== Framework Weights (A/B/C) ===")
print(json.dumps(out["framework_weights"], ensure_ascii=False, indent=2))

print("\n=== Ensemble Prior (ML2 입력 prior) ===")
print(json.dumps(out["ensemble_prior"], ensure_ascii=False, indent=2))
print("\nprior 합:", sum(out["ensemble_prior"].values()))

print("\n=== Explain ===")
print(json.dumps(out["explain"], ensure_ascii=False, indent=2))


=== AI2 Output (Context/Constraints/AI2 Weights) ===
{
  "project_context": {
    "urgency": 0.9,
    "ambiguity": 0.8,
    "external_collab": 0.9,
    "conflict_risk": 0.7,
    "innovation_need": 0.5
  },
  "constraints": {
    "team_size": 4,
    "must_include_dept": [],
    "max_moves_2y_avg": 5.0
  },
  "ai2_initial_weights": {
    "Trust": 0.14999985000015,
    "Communication": 0.24999975000025002,
    "Decision Making": 0.24999975000025002,
    "Innovative Thinking": 0.09999990000010002,
    "Psychological Safety": 0.14999985000015,
    "Conflict Management": 0.09999990000010002,
    "Role Definition": 9.99999000001e-07
  },
  "rationale": "본 프로젝트는 대외 협업이 많고 마감이 빠른 점에서 높은 긴급도와 외부 협력 지수가 부여되었습니다. 요구사항이 자주 바뀌고 이해관계자 조율이 필요해 불확실성과 갈등 위험도 역시 높게 평가하였습니다. 따라서 신속한 의사결정과 효과적인 커뮤니케이션이 핵심 경쟁력이며, 갈등 관리와 심리적 안전성 또한 중요한 요소입니다. 혁신 필요성은 명확히 언급되지 않아 중간 정도로 설정하였습니다. 제약조건은 현실적이며 팀 규모에 맞춰 최대 이직 수는 보통 수준으로 조정했습니다.",
  "confidence": 0.85
}

=== Framework Weights (A/B/C) ===
{
  "A": {
    "Trust": 0.

In [9]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Any, Tuple, Optional
import json
import random
from datetime import datetime

DRIVERS = [
    "Trust",
    "Communication",
    "Decision Making",
    "Innovative Thinking",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]

# -----------------------------
# 0) 입력 검증 유틸
# -----------------------------
def _assert_sum_to_one(w: Dict[str, float], tol: float = 1e-6) -> None:
    s = sum(float(v) for v in w.values())
    if abs(s - 1.0) > tol:
        raise ValueError(f"Weights must sum to 1.0, got {s}")

def _assert_drivers_complete(w: Dict[str, float]) -> None:
    missing = [d for d in DRIVERS if d not in w]
    extra = [k for k in w.keys() if k not in DRIVERS]
    if missing:
        raise ValueError(f"Missing drivers in weights: {missing}")
    if extra:
        raise ValueError(f"Unknown drivers in weights: {extra}")

# -----------------------------
# 1) Candidate(직원) 구조
# -----------------------------
@dataclass(frozen=True)
class Candidate:
    emp_id: str
    dept_tags: List[str]               # 예: ["Communication", "Decision Making"]
    moves_2y: float                    # 최근 2년 이동 횟수(평균/합 중 하나로 통일)
    driver_scores: Dict[str, float]    # 7 drivers 점수 (0~1)

# -----------------------------
# 2) 팀 점수(목적함수)
# -----------------------------
def team_score(team: List[Candidate], prior: Dict[str, float]) -> Tuple[float, Dict[str, float]]:
    avg = {d: sum(c.driver_scores[d] for c in team) / len(team) for d in DRIVERS}
    score = sum(prior[d] * avg[d] for d in DRIVERS)
    return score, avg

# -----------------------------
# 3) 제약조건 체크
# -----------------------------
def check_constraints(team: List[Candidate], constraints: Dict[str, Any]) -> bool:
    if len(team) != int(constraints["team_size"]):
        return False

    must = set(constraints.get("must_include_dept", []))
    team_tags = set(tag for c in team for tag in c.dept_tags)
    if not must.issubset(team_tags):
        return False

    max_moves = constraints.get("max_moves_2y_avg", None)
    if max_moves is not None:
        avg_moves = sum(c.moves_2y for c in team) / len(team)
        if avg_moves > float(max_moves):
            return False

    return True

# -----------------------------
# 4) 최적화 엔진 (MVP: 랜덤 + 힐클라임)
# -----------------------------
def optimize_team(
    candidates: List[Candidate],
    prior: Dict[str, float],
    constraints: Dict[str, Any],
    top_k: int = 3,
    n_random: int = 3000,
    n_hillclimb_steps: int = 400,
    seed: int = 42,
) -> List[Dict[str, Any]]:
    random.seed(seed)
    team_size = int(constraints["team_size"])

    best: List[Tuple[float, List[Candidate], Dict[str, float]]] = []

    def push_best(sc: float, team: List[Candidate], avg: Dict[str, float]):
        nonlocal best
        best.append((sc, team, avg))
        best.sort(key=lambda x: x[0], reverse=True)
        best = best[:top_k]

    feasible: List[List[Candidate]] = []

    # 4-1) 랜덤 feasible 팀 탐색
    for _ in range(n_random):
        team = random.sample(candidates, team_size)
        if check_constraints(team, constraints):
            feasible.append(team)
            sc, avg = team_score(team, prior)
            push_best(sc, team, avg)

    if not feasible:
        raise ValueError("No feasible team found. Relax constraints or increase candidate pool.")

    # 4-2) 간단 힐클라임
    for _ in range(n_hillclimb_steps):
        team = random.choice(feasible)
        current_sc, _ = team_score(team, prior)

        out_idx = random.randrange(team_size)
        remaining = team[:out_idx] + team[out_idx + 1 :]

        pool = [c for c in candidates if c not in team]
        in_member = random.choice(pool)
        new_team = remaining + [in_member]

        if not check_constraints(new_team, constraints):
            continue

        new_sc, new_avg = team_score(new_team, prior)
        if new_sc > current_sc:
            feasible.append(new_team)
            push_best(new_sc, new_team, new_avg)

    results = []
    for rank, (sc, team, avg) in enumerate(best, start=1):
        contrib = {d: prior[d] * avg[d] for d in DRIVERS}
        results.append({
            "rank": rank,
            "score": round(sc, 4),
            "team": [c.emp_id for c in team],
            "avg_driver_scores": {d: round(avg[d], 4) for d in DRIVERS},
            "contribution": {d: round(contrib[d], 4) for d in DRIVERS},
            "constraints_ok": True,
        })
    return results

# -----------------------------
# 5) History DB (JSONL)
# -----------------------------
def log_event(path: str, event: Dict[str, Any]) -> None:
    event = dict(event)
    event["ts"] = datetime.utcnow().isoformat() + "Z"
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(event, ensure_ascii=False) + "\n")

# -----------------------------
# 6) Online Update (EMA 방식)
# -----------------------------
def update_prior_ema(
    prior: Dict[str, float],
    team_avg: Dict[str, float],
    feedback: Dict[str, Any],
    lr: float = 0.05,
) -> Dict[str, float]:
    accepted = bool(feedback.get("accepted", False))
    outcome_score = float(feedback.get("outcome_score", 0.5))  # 0~1

    strength = (1.0 if accepted else 0.3) * (0.2 + 0.8 * outcome_score)
    step = lr * strength

    new_prior = {}
    for d in DRIVERS:
        new_prior[d] = (1 - step) * prior[d] + step * team_avg[d]

    s = sum(new_prior.values())
    new_prior = {d: new_prior[d] / s for d in DRIVERS}
    return new_prior

# -----------------------------
# 7) ML2 Runner: 너 JSON을 받아 실행
# -----------------------------
def run_ml2(ai2_output: Dict[str, Any], ensemble_prior: Dict[str, float], candidates: List[Candidate]) -> Dict[str, Any]:
    constraints = ai2_output["constraints"]
    context = ai2_output["project_context"]

    # validate prior
    _assert_drivers_complete(ensemble_prior)
    _assert_sum_to_one(ensemble_prior)

    # optimize
    top_teams = optimize_team(candidates, ensemble_prior, constraints, top_k=3)

    return {
        "context": context,
        "constraints": constraints,
        "prior_used": ensemble_prior,
        "recommendations": top_teams
    }

# -----------------------------
# 8) 데모: 후보자 없을 때 더미 생성해서라도 실행
# -----------------------------
def make_dummy_candidates(n: int = 40, seed: int = 7) -> List[Candidate]:
    random.seed(seed)

    tag_pool = [
        ["Communication"],
        ["Decision Making"],
        ["Conflict Management"],
        ["Role Definition"],
        ["Innovative Thinking"],
        ["Psychological Safety"],
        ["Trust"],
        ["Communication", "Decision Making"],  # 복합 태그도 일부
    ]

    candidates: List[Candidate] = []
    for i in range(1, n + 1):
        scores = {d: random.random() for d in DRIVERS}  # 0~1
        tags = random.choice(tag_pool)
        moves = random.uniform(0, 8)
        candidates.append(Candidate(emp_id=f"E{i:03d}", dept_tags=tags, moves_2y=moves, driver_scores=scores))
    return candidates

if __name__ == "__main__":
    # ---- 너가 준 입력을 그대로 변수로 둠 ----
    ai2_output = {
        "project_context": {"urgency": 0.9, "ambiguity": 0.8, "external_collab": 0.9, "conflict_risk": 0.7, "innovation_need": 0.5},
        "constraints": {"team_size": 6, "must_include_dept": ["Communication", "Decision Making"], "max_moves_2y_avg": 5.0},
        "ai2_initial_weights": {
            "Trust": 0.12962962962962962,
            "Communication": 0.1851851851851852,
            "Decision Making": 0.1851851851851852,
            "Innovative Thinking": 0.11111111111111112,
            "Psychological Safety": 0.12962962962962962,
            "Conflict Management": 0.14814814814814817,
            "Role Definition": 0.11111111111111112
        },
        "confidence": 0.85
    }

    ensemble_prior = {
        "Trust": 0.13828329453259547,
        "Communication": 0.18402661043895163,
        "Decision Making": 0.1719252474507434,
        "Innovative Thinking": 0.10418710476211979,
        "Psychological Safety": 0.1317139366432381,
        "Conflict Management": 0.1359299234162376,
        "Role Definition": 0.1339338827561141
    }

    candidates = make_dummy_candidates()

    output = run_ml2(ai2_output, ensemble_prior, candidates)
    print(json.dumps(output, ensure_ascii=False, indent=2))

    # ---- (선택) History + Update까지 데모 ----
    history_path = "ml2_history.jsonl"
    chosen = output["recommendations"][0]  # 1등 팀 선택 가정
    feedback = {"accepted": True, "outcome_score": 0.8}

    log_event(history_path, {
        "context": output["context"],
        "constraints": output["constraints"],
        "prior_before": ensemble_prior,
        "recommendation": chosen,
        "feedback": feedback
    })

    updated_prior = update_prior_ema(ensemble_prior, chosen["avg_driver_scores"], feedback, lr=0.05)
    print("\n=== updated_prior ===")
    print(json.dumps(updated_prior, ensure_ascii=False, indent=2))


{
  "context": {
    "urgency": 0.9,
    "ambiguity": 0.8,
    "external_collab": 0.9,
    "conflict_risk": 0.7,
    "innovation_need": 0.5
  },
  "constraints": {
    "team_size": 6,
    "must_include_dept": [
      "Communication",
      "Decision Making"
    ],
    "max_moves_2y_avg": 5.0
  },
  "prior_used": {
    "Trust": 0.13828329453259547,
    "Communication": 0.18402661043895163,
    "Decision Making": 0.1719252474507434,
    "Innovative Thinking": 0.10418710476211979,
    "Psychological Safety": 0.1317139366432381,
    "Conflict Management": 0.1359299234162376,
    "Role Definition": 0.1339338827561141
  },
  "recommendations": [
    {
      "rank": 1,
      "score": 0.6048,
      "team": [
        "E035",
        "E028",
        "E017",
        "E033",
        "E019",
        "E009"
      ],
      "avg_driver_scores": {
        "Trust": 0.7499,
        "Communication": 0.7984,
        "Decision Making": 0.5056,
        "Innovative Thinking": 0.5409,
        "Psychological Sa

In [10]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Any, List, Optional
import json
from datetime import datetime
import math

DRIVERS = [
    "Trust",
    "Communication",
    "Decision Making",
    "Innovative Thinking",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]

# -----------------------------
# 0) 유틸: 검증/정규화
# -----------------------------
def validate_weights(w: Dict[str, float], name: str) -> None:
    missing = [d for d in DRIVERS if d not in w]
    extra = [k for k in w.keys() if k not in DRIVERS]
    if missing:
        raise ValueError(f"{name}: missing drivers {missing}")
    if extra:
        raise ValueError(f"{name}: unknown drivers {extra}")

def normalize(w: Dict[str, float]) -> Dict[str, float]:
    s = sum(float(v) for v in w.values())
    if s <= 0:
        raise ValueError("Cannot normalize: sum <= 0")
    return {k: float(v) / s for k, v in w.items()}

def assert_sum_to_one(w: Dict[str, float], tol: float = 1e-6) -> None:
    s = sum(w.values())
    if abs(s - 1.0) > tol:
        raise ValueError(f"Weights must sum to 1.0, got {s}")

# -----------------------------
# 1) ML2: Ensemble prior 생성
# -----------------------------
def compute_ensemble_prior(
    ai2_initial: Dict[str, float],
    fw_a: Dict[str, float],
    fw_b: Dict[str, float],
    fw_c: Dict[str, float],
    alpha: Dict[str, float],
) -> Dict[str, float]:
    """
    ensemble_prior = α_AI2 * ai2 + α_A * A + α_B * B + α_C * C
    """
    for name, w in [("AI2", ai2_initial), ("A", fw_a), ("B", fw_b), ("C", fw_c)]:
        validate_weights(w, name)

    # alpha 검증
    needed = {"AI2", "A", "B", "C"}
    if set(alpha.keys()) != needed:
        raise ValueError(f"alpha must have keys {needed}, got {set(alpha.keys())}")
    alpha = {k: float(v) for k, v in alpha.items()}
    a_sum = sum(alpha.values())
    if not math.isclose(a_sum, 1.0, rel_tol=0, abs_tol=1e-9):
        # alpha가 1이 아니어도 정규화해서 안전하게 처리
        alpha = {k: v / a_sum for k, v in alpha.items()}

    prior = {}
    for d in DRIVERS:
        prior[d] = (
            alpha["AI2"] * float(ai2_initial[d])
            + alpha["A"] * float(fw_a[d])
            + alpha["B"] * float(fw_b[d])
            + alpha["C"] * float(fw_c[d])
        )

    prior = normalize(prior)
    assert_sum_to_one(prior)
    return prior

# -----------------------------
# 2) History DB: JSONL 저장
# -----------------------------
def log_ml2_event(path: str, event: Dict[str, Any]) -> None:
    event = dict(event)
    event["ts"] = datetime.utcnow().isoformat() + "Z"
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(event, ensure_ascii=False) + "\n")

# -----------------------------
# 3) ML2: Online update (prior 업데이트)
# -----------------------------
def update_prior_from_feedback(
    prior: Dict[str, float],
    feedback: Dict[str, Any],
    lr: float = 0.05,
) -> Dict[str, float]:
    """
    "통합 매칭 엔진"이 없으니 팀 평균(team_avg) 같은 신호가 없다.
    대신, 피드백에서 "어떤 드라이버가 문제였는지/좋았는지"를 태그로 받는 최소 설계.

    feedback 예시:
      {
        "accepted": true,
        "outcome_score": 0.8,
        "boost_drivers": ["Communication", "Decision Making"],
        "penalize_drivers": ["Conflict Management"]
      }

    - boost는 prior를 조금 올리고
    - penalize는 prior를 조금 내린다
    """
    validate_weights(prior, "prior")
    prior = dict(prior)

    accepted = bool(feedback.get("accepted", False))
    outcome = float(feedback.get("outcome_score", 0.5))  # 0~1
    boost = set(feedback.get("boost_drivers", []))
    penal = set(feedback.get("penalize_drivers", []))

    # 업데이트 강도: 채택+성과 좋으면 세게, 아니면 약하게
    strength = (1.0 if accepted else 0.3) * (0.2 + 0.8 * outcome)  # 0.06~1.0
    step = lr * strength

    # prior를 "조금" 이동시키기 위한 target 벡터 생성
    target = {d: prior[d] for d in DRIVERS}

    for d in boost:
        if d in target:
            target[d] += 1.0  # boost 신호
    for d in penal:
        if d in target:
            target[d] = max(0.0, target[d] - 0.5)  # penal 신호

    target = normalize(target)

    new_prior = {}
    for d in DRIVERS:
        new_prior[d] = (1 - step) * prior[d] + step * target[d]

    new_prior = normalize(new_prior)
    assert_sum_to_one(new_prior)
    return new_prior

# -----------------------------
# 4) ML2 Runner (엔진 없이 ML2만)
# -----------------------------
def run_ml2_only(payload: Dict[str, Any], history_path: str = "ml2_history.jsonl") -> Dict[str, Any]:
    ai2 = payload["ai2_output"]
    fw = payload["framework_weights"]
    explain = payload["explain"]

    alpha = explain["alpha"]  # {"AI2":0.4,"A":0.2,"B":0.2,"C":0.2}

    prior = compute_ensemble_prior(
        ai2_initial=ai2["ai2_initial_weights"],
        fw_a=fw["A"],
        fw_b=fw["B"],
        fw_c=fw["C"],
        alpha=alpha
    )

    # History에 "prior 생성 이벤트" 기록
    log_ml2_event(history_path, {
        "event_type": "prior_generated",
        "project_context": ai2.get("project_context", {}),
        "constraints": ai2.get("constraints", {}),
        "alpha": alpha,
        "prior": prior,
        "rationale": ai2.get("rationale"),
        "confidence": ai2.get("confidence"),
    })

    return {"ensemble_prior": prior}

# -----------------------------
# 5) 데모 실행
# -----------------------------
if __name__ == "__main__":
    # 네가 준 값들을 그대로 넣을 수 있도록 payload 형태로 정리
    payload = {
        "ai2_output": {
            "project_context": {"urgency": 0.9, "ambiguity": 0.8, "external_collab": 0.9, "conflict_risk": 0.7, "innovation_need": 0.5},
            "constraints": {"team_size": 6, "must_include_dept": ["Communication", "Decision Making"], "max_moves_2y_avg": 5.0},
            "ai2_initial_weights": {
                "Trust": 0.12962962962962962,
                "Communication": 0.1851851851851852,
                "Decision Making": 0.1851851851851852,
                "Innovative Thinking": 0.11111111111111112,
                "Psychological Safety": 0.12962962962962962,
                "Conflict Management": 0.14814814814814817,
                "Role Definition": 0.11111111111111112
            },
            "rationale": "…",
            "confidence": 0.85
        },
        "framework_weights": {
            "A": { "Trust": 0.12184249628528974, "Communication": 0.20802377414561662, "Decision Making": 0.18127786032689452,
                   "Innovative Thinking": 0.1263001485884101, "Psychological Safety": 0.1337295690936107,
                   "Conflict Management": 0.07429420505200594, "Role Definition": 0.15453194650817237 },
            "B": { "Trust": 0.17391304347826086, "Communication": 0.15261756876663712, "Decision Making": 0.08873114463176575,
                   "Innovative Thinking": 0.08873114463176575, "Psychological Safety": 0.18189884649511978,
                   "Conflict Management": 0.22537710736468503, "Role Definition": 0.08873114463176575 },
            "C": { "Trust": 0.13640167364016736, "Communication": 0.1891213389121339, "Decision Making": 0.2192468619246862,
                   "Innovative Thinking": 0.08368200836820085, "Psychological Safety": 0.08368200836820085,
                   "Conflict Management": 0.08368200836820085, "Role Definition": 0.2041841004184101 }
        },
        "explain": {
            "alpha": {"AI2": 0.4, "A": 0.2, "B": 0.2, "C": 0.2}
        }
    }

    out = run_ml2_only(payload)
    print("=== ensemble_prior ===")
    print(json.dumps(out["ensemble_prior"], ensure_ascii=False, indent=2))

    # (선택) 피드백으로 prior 업데이트 데모
    feedback = {
        "accepted": True,
        "outcome_score": 0.8,
        "boost_drivers": ["Communication", "Decision Making"],
        "penalize_drivers": ["Innovative Thinking"]
    }
    updated = update_prior_from_feedback(out["ensemble_prior"], feedback, lr=0.05)
    log_ml2_event("ml2_history.jsonl", {"event_type": "prior_updated", "feedback": feedback, "prior_after": updated})
    print("\n=== prior_updated ===")
    print(json.dumps(updated, ensure_ascii=False, indent=2))


=== ensemble_prior ===
{
  "Trust": 0.13828329453259547,
  "Communication": 0.18402661043895163,
  "Decision Making": 0.1719252474507434,
  "Innovative Thinking": 0.10418710476211979,
  "Psychological Safety": 0.1317139366432381,
  "Conflict Management": 0.1359299234162376,
  "Role Definition": 0.1339338827561141
}

=== prior_updated ===
{
  "Trust": 0.13448101551005104,
  "Communication": 0.19347025894166198,
  "Decision Making": 0.18170163866680739,
  "Innovative Thinking": 0.09981124636211075,
  "Psychological Safety": 0.12809229065940395,
  "Conflict Management": 0.13219235339312968,
  "Role Definition": 0.13025119646683525
}


In [11]:
from __future__ import annotations
from dataclasses import dataclass, asdict
from typing import Dict, List, Any, Optional, Tuple
import json
from datetime import datetime

DRIVERS = [
    "Trust",
    "Communication",
    "Decision Making",
    "Innovative Thinking",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]


# ============================
# 1) Input/Output Schemas
# ============================
@dataclass
class RecommendationEvent:
    """
    최적화/매칭 엔진이 만든 '최종 산출물'을 ML2/History로 넘기기 직전 이벤트.
    (팀 자체는 여기 있거나 없어도 되지만, 보통 recommendation_id로 연결)
    """
    recommendation_id: str
    project_id: str
    project_context: Dict[str, float]      # urgency/ambiguity/external_collab/conflict_risk/innovation_need
    constraints: Dict[str, Any]
    prior_used: Dict[str, float]           # ML2가 만든 prior
    explanation: Optional[Dict[str, Any]] = None  # (선택) 기여도/근거 등


@dataclass
class UserFeedback:
    """
    사용자(HR)가 최종 산출물에 대해 보인 반응.
    """
    accepted: bool                         # 채택/거절
    modifications_count: int = 0           # 몇 명 교체했는지(0이면 그대로 채택)
    comment_tags: Optional[List[str]] = None   # 예: ["needs_faster_decisions", "collab_issue"]
    boost_drivers: Optional[List[str]] = None  # 사용자가 직접 선택한 강화 드라이버(선택)
    penalize_drivers: Optional[List[str]] = None  # 사용자가 직접 선택한 약화 드라이버(선택)


@dataclass
class ProjectKPI:
    """
    프로젝트가 끝났거나 중간 체크포인트에서 수집하는 운영/성과 지표.
    0~1로 정규화된 값으로 넣는 걸 추천(없으면 None 허용).
    """
    on_time: Optional[float] = None            # 일정 준수(1=완전 준수)
    satisfaction: Optional[float] = None       # 팀/PM 만족도(1=최고)
    conflict_incidents: Optional[int] = None   # 갈등/에스컬레이션 건수(정수)
    rework_hours: Optional[float] = None       # 재작업 시간(시간)
    output_quality: Optional[float] = None     # 산출물 품질(1=최고)


@dataclass
class EvaluatedOutcome:
    """
    Performance Evaluation / Outcome Interpretation 레이어의 산출물.
    History DB에 저장되는 '학습 가능한 사건(event)' 형태.
    """
    event_type: str                           # "evaluated_outcome"
    recommendation_id: str
    project_id: str
    outcome_score: float                      # 0~1
    accepted: bool
    modifications_count: int
    boost_drivers: List[str]
    penalize_drivers: List[str]
    signals: Dict[str, Any]                   # 계산에 쓰인 세부 신호/중간값
    ts: str                                   # ISO timestamp


# ============================
# 2) Helper: normalize / clamp
# ============================
def clamp01(x: float) -> float:
    return 0.0 if x < 0 else 1.0 if x > 1 else x


def safe_mean(values: List[Optional[float]]) -> Optional[float]:
    xs = [v for v in values if v is not None]
    if not xs:
        return None
    return sum(xs) / len(xs)


# ============================
# 3) Evaluation Logic
# ============================
def evaluate_outcome(
    rec: RecommendationEvent,
    feedback: UserFeedback,
    kpi: Optional[ProjectKPI] = None,
    policy: Optional[Dict[str, Any]] = None,
) -> EvaluatedOutcome:
    """
    이 레이어의 핵심:
    - 최종 산출물(추천) + 사용자 반응 + KPI를
      "학습 가능한 outcome_score + driver 태그(boost/penalize)"로 변환

    policy로 가중치/패널티 튜닝 가능.
    """
    policy = policy or {}

    # --- (A) 기본 점수: 사용자 채택/수정 기반 ---
    # 채택이 가장 강한 신호. 수정이 많으면 "추천 품질"이 낮았다고 가정.
    accept_score = 1.0 if feedback.accepted else 0.0

    # 수정 패널티: 팀 크기에 비례해 정규화(0~1)
    team_size = float(rec.constraints.get("team_size", 6))
    mod_ratio = clamp01(feedback.modifications_count / max(team_size, 1.0))
    mod_penalty = policy.get("mod_penalty_weight", 0.35) * mod_ratio  # 기본 0.35

    # 사용자 기반 중간 점수
    user_score = clamp01(accept_score - mod_penalty)

    # --- (B) KPI 기반 점수: 있으면 반영 ---
    # KPI가 없으면 user_score만으로도 돌아가게 설계.
    kpi_score = None
    signals: Dict[str, Any] = {
        "accept_score": accept_score,
        "mod_ratio": mod_ratio,
        "mod_penalty": mod_penalty,
        "user_score": user_score,
    }

    if kpi is not None:
        # conflict_incidents / rework_hours는 낮을수록 좋으니 0~1로 변환
        # (여기선 간단히 cap을 두고 역정규화; 조직마다 캡은 정책으로 조정)
        conflict_cap = policy.get("conflict_cap", 5)  # 5건 이상이면 최악으로 취급
        rework_cap = policy.get("rework_cap", 40.0)   # 40시간 이상이면 최악

        conflict_norm = None
        if kpi.conflict_incidents is not None:
            conflict_norm = 1.0 - clamp01(kpi.conflict_incidents / conflict_cap)

        rework_norm = None
        if kpi.rework_hours is not None:
            rework_norm = 1.0 - clamp01(kpi.rework_hours / rework_cap)

        # KPI 요소 평균 (있을 때만)
        kpi_score = safe_mean([
            kpi.on_time,
            kpi.satisfaction,
            conflict_norm,
            rework_norm,
            kpi.output_quality
        ])

        signals.update({
            "kpi_on_time": kpi.on_time,
            "kpi_satisfaction": kpi.satisfaction,
            "kpi_conflict_norm": conflict_norm,
            "kpi_rework_norm": rework_norm,
            "kpi_output_quality": kpi.output_quality,
            "kpi_score": kpi_score,
        })

    # --- (C) 최종 outcome_score 결합 ---
    # KPI가 없으면 user_score를 그대로 쓰고,
    # KPI가 있으면 user_score와 섞는다.
    user_w = policy.get("user_weight", 0.6)   # 기본: 사용자 반응을 더 크게
    kpi_w = 1.0 - user_w

    if kpi_score is None:
        outcome_score = user_score
        signals["outcome_mix"] = "user_only"
    else:
        outcome_score = clamp01(user_w * user_score + kpi_w * kpi_score)
        signals["outcome_mix"] = {"user_weight": user_w, "kpi_weight": kpi_w}

    # --- (D) driver 태그 생성 (boost/penalize) ---
    # 1) 사용자가 직접 준 태그가 있으면 우선 반영
    boost = set(d for d in (feedback.boost_drivers or []) if d in DRIVERS)
    penal = set(d for d in (feedback.penalize_drivers or []) if d in DRIVERS)

    # 2) 없으면 "자동 규칙"으로 만든다: context와 KPI 신호로 추론
    # (너희가 이미 프레임워크/AI2로 해온 방식과 톤 일치)
    if not boost and not penal:
        ctx = rec.project_context or {}
        urgency = ctx.get("urgency", 0.0)
        ambiguity = ctx.get("ambiguity", 0.0)
        external = ctx.get("external_collab", 0.0)
        conflict_risk = ctx.get("conflict_risk", 0.0)

        # 성과가 좋으면: 현재 상위 prior 드라이버를 강화(Top-2)
        if outcome_score >= policy.get("good_threshold", 0.75):
            top2 = sorted(rec.prior_used.items(), key=lambda x: x[1], reverse=True)[:2]
            boost.update([k for k, _ in top2])

        # 성과가 나쁘면: 상황에 따라 penalize 후보 선정
        if outcome_score <= policy.get("bad_threshold", 0.45):
            if (urgency >= 0.7) or (ambiguity >= 0.7):
                penal.add("Decision Making")
                penal.add("Role Definition")
            if external >= 0.7:
                penal.add("Communication")
            if conflict_risk >= 0.6:
                penal.add("Conflict Management")
                penal.add("Psychological Safety")

        # KPI 기반 보정: 갈등이 실제로 많았으면 갈등/심리안전을 boost
        if kpi is not None and kpi.conflict_incidents is not None and kpi.conflict_incidents >= 2:
            boost.add("Conflict Management")
            boost.add("Psychological Safety")

    # 충돌 정리: 같은 드라이버가 boost와 penal에 동시에 있으면 제거(중립)
    overlap = boost.intersection(penal)
    boost.difference_update(overlap)
    penal.difference_update(overlap)

    evaluated = EvaluatedOutcome(
        event_type="evaluated_outcome",
        recommendation_id=rec.recommendation_id,
        project_id=rec.project_id,
        outcome_score=float(outcome_score),
        accepted=bool(feedback.accepted),
        modifications_count=int(feedback.modifications_count),
        boost_drivers=sorted(boost),
        penalize_drivers=sorted(penal),
        signals=signals,
        ts=datetime.utcnow().isoformat() + "Z",
    )
    return evaluated


# ============================
# 4) History DB writer (JSONL)
# ============================
def write_history_jsonl(path: str, outcome: EvaluatedOutcome) -> None:
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(asdict(outcome), ensure_ascii=False) + "\n")


# ============================
# 5) Demo run
# ============================
if __name__ == "__main__":
    # (예시) 너희가 가진 ML2 prior / context / constraints
    rec = RecommendationEvent(
        recommendation_id="REC-0001",
        project_id="PRJ-2026-01",
        project_context={
            "urgency": 0.9,
            "ambiguity": 0.8,
            "external_collab": 0.9,
            "conflict_risk": 0.7,
            "innovation_need": 0.5,
        },
        constraints={
            "team_size": 6,
            "must_include_dept": ["Communication", "Decision Making"],
            "max_moves_2y_avg": 5.0,
        },
        prior_used={
            "Trust": 0.13828329453259547,
            "Communication": 0.18402661043895163,
            "Decision Making": 0.1719252474507434,
            "Innovative Thinking": 0.10418710476211979,
            "Psychological Safety": 0.1317139366432381,
            "Conflict Management": 0.1359299234162376,
            "Role Definition": 0.1339338827561141,
        },
        explanation={"note": "Optimizer output stored separately"},
    )

    # (예시) 사용자 피드백: 채택 + 수정 1명 + 드라이버 태그(선택)
    feedback = UserFeedback(
        accepted=True,
        modifications_count=1,
        boost_drivers=["Communication", "Decision Making"],
        penalize_drivers=["Innovative Thinking"],
    )

    # (예시) KPI(선택): 없으면 None으로 둬도 됨
    kpi = ProjectKPI(
        on_time=0.9,
        satisfaction=0.8,
        conflict_incidents=1,
        rework_hours=8.0,
        output_quality=0.85,
    )

    outcome = evaluate_outcome(rec, feedback, kpi)
    print(json.dumps(asdict(outcome), ensure_ascii=False, indent=2))

    write_history_jsonl("history_events.jsonl", outcome)


{
  "event_type": "evaluated_outcome",
  "recommendation_id": "REC-0001",
  "project_id": "PRJ-2026-01",
  "outcome_score": 0.8969999999999999,
  "accepted": true,
  "modifications_count": 1,
  "boost_drivers": [
    "Communication",
    "Decision Making"
  ],
  "penalize_drivers": [
    "Innovative Thinking"
  ],
  "signals": {
    "accept_score": 1.0,
    "mod_ratio": 0.16666666666666666,
    "mod_penalty": 0.05833333333333333,
    "user_score": 0.9416666666666667,
    "kpi_on_time": 0.9,
    "kpi_satisfaction": 0.8,
    "kpi_conflict_norm": 0.8,
    "kpi_rework_norm": 0.8,
    "kpi_output_quality": 0.85,
    "kpi_score": 0.8299999999999998,
    "outcome_mix": {
      "user_weight": 0.6,
      "kpi_weight": 0.4
    }
  },
  "ts": "2026-01-27T15:24:18.467099Z"
}


In [5]:
import numpy as np
import math
import itertools
import heapq
from dataclasses import dataclass
from typing import List, Dict, Tuple

# ----------------------------
# 1) 데이터 구조
# ----------------------------
@dataclass(frozen=True)
class Employee:
    name: str
    trust: float
    communication: float
    decision_making: float
    innovative_thinking: float
    conflict_management: float
    role_clarity: float
    psychological_safety: float

_METRICS = [
    "trust", "communication", "decision_making",
    "innovative_thinking", "conflict_management",
    "role_clarity", "psychological_safety"
]

_DRIVER_KO = {
    "trust": "신뢰",
    "communication": "커뮤니케이션",
    "innovative_thinking": "혁신/아이디어",
    "decision_making": "의사결정 균형",
    "conflict_management": "갈등관리",
    "role_clarity": "역할명확성",
    "psychological_safety": "심리적 안전",
}


 #샘플 데이터: 50명 (E001~E050)
# ----------------------------
SAMPLE_EMPLOYEES_50: List[Employee] = [
    Employee(name="E001", trust=0.82, communication=0.78, decision_making=0.52, innovative_thinking=0.54, conflict_management=0.76, role_clarity=0.88, psychological_safety=0.73),
    Employee(name="E002", trust=0.80, communication=0.74, decision_making=0.48, innovative_thinking=0.50, conflict_management=0.72, role_clarity=0.86, psychological_safety=0.70),
    Employee(name="E003", trust=0.85, communication=0.81, decision_making=0.55, innovative_thinking=0.57, conflict_management=0.79, role_clarity=0.90, psychological_safety=0.76),
    Employee(name="E004", trust=0.78, communication=0.72, decision_making=0.44, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.84, psychological_safety=0.68),
    Employee(name="E005", trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79),
    Employee(name="E006", trust=0.76, communication=0.70, decision_making=0.46, innovative_thinking=0.47, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.66),
    Employee(name="E007", trust=0.84, communication=0.79, decision_making=0.50, innovative_thinking=0.56, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),
    Employee(name="E008", trust=0.81, communication=0.76, decision_making=0.53, innovative_thinking=0.52, conflict_management=0.75, role_clarity=0.87, psychological_safety=0.71),
    Employee(name="E009", trust=0.79, communication=0.73, decision_making=0.49, innovative_thinking=0.51, conflict_management=0.72, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E010", trust=0.86, communication=0.82, decision_making=0.56, innovative_thinking=0.58, conflict_management=0.80, role_clarity=0.91, psychological_safety=0.77),
    Employee(name="E011", trust=0.83, communication=0.77, decision_making=0.47, innovative_thinking=0.53, conflict_management=0.74, role_clarity=0.88, psychological_safety=0.72),
    Employee(name="E012", trust=0.77, communication=0.71, decision_making=0.45, innovative_thinking=0.48, conflict_management=0.70, role_clarity=0.84, psychological_safety=0.67),
    Employee(name="E013", trust=0.87, communication=0.84, decision_making=0.57, innovative_thinking=0.59, conflict_management=0.81, role_clarity=0.92, psychological_safety=0.78),
    Employee(name="E014", trust=0.82, communication=0.75, decision_making=0.51, innovative_thinking=0.55, conflict_management=0.76, role_clarity=0.87, psychological_safety=0.70),
    Employee(name="E015", trust=0.80, communication=0.74, decision_making=0.43, innovative_thinking=0.46, conflict_management=0.73, role_clarity=0.86, psychological_safety=0.68),
    Employee(name="E016", trust=0.85, communication=0.80, decision_making=0.54, innovative_thinking=0.58, conflict_management=0.78, role_clarity=0.90, psychological_safety=0.75),
    Employee(name="E017", trust=0.78, communication=0.72, decision_making=0.50, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E018", trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80),
    Employee(name="E019", trust=0.76, communication=0.70, decision_making=0.47, innovative_thinking=0.45, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.65),
    Employee(name="E020", trust=0.84, communication=0.79, decision_making=0.53, innovative_thinking=0.57, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),

    Employee(name="E021", trust=0.70, communication=0.62, decision_making=0.30, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.52, psychological_safety=0.86),
    Employee(name="E022", trust=0.66, communication=0.58, decision_making=0.78, innovative_thinking=0.90, conflict_management=0.53, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E023", trust=0.72, communication=0.70, decision_making=0.45, innovative_thinking=0.84, conflict_management=0.60, role_clarity=0.58, psychological_safety=0.80),
    Employee(name="E024", trust=0.60, communication=0.55, decision_making=0.82, innovative_thinking=0.92, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E025", trust=0.75, communication=0.68, decision_making=0.38, innovative_thinking=0.86, conflict_management=0.62, role_clarity=0.60, psychological_safety=0.83),
    Employee(name="E026", trust=0.63, communication=0.57, decision_making=0.70, innovative_thinking=0.89, conflict_management=0.54, role_clarity=0.50, psychological_safety=0.88),
    Employee(name="E027", trust=0.78, communication=0.73, decision_making=0.55, innovative_thinking=0.81, conflict_management=0.68, role_clarity=0.63, psychological_safety=0.78),
    Employee(name="E028", trust=0.65, communication=0.60, decision_making=0.25, innovative_thinking=0.90, conflict_management=0.52, role_clarity=0.47, psychological_safety=0.92),
    Employee(name="E029", trust=0.69, communication=0.64, decision_making=0.66, innovative_thinking=0.87, conflict_management=0.56, role_clarity=0.54, psychological_safety=0.85),
    Employee(name="E030", trust=0.74, communication=0.71, decision_making=0.48, innovative_thinking=0.83, conflict_management=0.61, role_clarity=0.59, psychological_safety=0.81),
    Employee(name="E031", trust=0.62, communication=0.56, decision_making=0.80, innovative_thinking=0.91, conflict_management=0.49, role_clarity=0.45, psychological_safety=0.90),
    Employee(name="E032", trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82),
    Employee(name="E033", trust=0.68, communication=0.63, decision_making=0.60, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.53, psychological_safety=0.87),
    Employee(name="E034", trust=0.61, communication=0.55, decision_making=0.74, innovative_thinking=0.92, conflict_management=0.51, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E035", trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79),
    Employee(name="E036", trust=0.64, communication=0.59, decision_making=0.83, innovative_thinking=0.90, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E037", trust=0.71, communication=0.66, decision_making=0.58, innovative_thinking=0.86, conflict_management=0.57, role_clarity=0.55, psychological_safety=0.84),
    Employee(name="E038", trust=0.67, communication=0.61, decision_making=0.28, innovative_thinking=0.89, conflict_management=0.52, role_clarity=0.49, psychological_safety=0.92),
    Employee(name="E039", trust=0.73, communication=0.69, decision_making=0.72, innovative_thinking=0.82, conflict_management=0.60, role_clarity=0.57, psychological_safety=0.80),
    Employee(name="E040", trust=0.65, communication=0.60, decision_making=0.50, innovative_thinking=0.87, conflict_management=0.54, role_clarity=0.51, psychological_safety=0.88),

    Employee(name="E041", trust=0.62, communication=0.70, decision_making=0.88, innovative_thinking=0.68, conflict_management=0.55, role_clarity=0.50, psychological_safety=0.60),
    Employee(name="E042", trust=0.58, communication=0.66, decision_making=0.92, innovative_thinking=0.72, conflict_management=0.50, role_clarity=0.48, psychological_safety=0.58),
    Employee(name="E043", trust=0.66, communication=0.73, decision_making=0.80, innovative_thinking=0.65, conflict_management=0.60, role_clarity=0.52, psychological_safety=0.62),
    Employee(name="E044", trust=0.55, communication=0.61, decision_making=0.95, innovative_thinking=0.75, conflict_management=0.48, role_clarity=0.45, psychological_safety=0.55),
    Employee(name="E045", trust=0.70, communication=0.76, decision_making=0.78, innovative_thinking=0.63, conflict_management=0.62, role_clarity=0.55, psychological_safety=0.65),
    Employee(name="E046", trust=0.60, communication=0.68, decision_making=0.85, innovative_thinking=0.70, conflict_management=0.53, role_clarity=0.49, psychological_safety=0.59),
    Employee(name="E047", trust=0.64, communication=0.72, decision_making=0.90, innovative_thinking=0.74, conflict_management=0.56, role_clarity=0.51, psychological_safety=0.61),
    Employee(name="E048", trust=0.57, communication=0.63, decision_making=0.82, innovative_thinking=0.67, conflict_management=0.49, role_clarity=0.46, psychological_safety=0.56),
    Employee(name="E049", trust=0.71, communication=0.78, decision_making=0.76, innovative_thinking=0.62, conflict_management=0.63, role_clarity=0.56, psychological_safety=0.67),
    Employee(name="E050", trust=0.59, communication=0.67, decision_making=0.93, innovative_thinking=0.73, conflict_management=0.52, role_clarity=0.48, psychological_safety=0.58),
]

# ----------------------------
# 2) 기본 함수
# ----------------------------
def mean(vals) -> float:
    return float(np.mean(vals))

def diversity(vals) -> float:
    return float(np.std(vals))

def balance_score(std: float, target: float, tolerance: float) -> float:
    return float(np.exp(-((std - target) ** 2) / (2 * (tolerance ** 2))))

# ----------------------------
# 3) weights 검증 (외부 주입용 안전장치)
# ----------------------------
def validate_weights(weights: Dict[str, float], require_sum_to_one: bool = True) -> Dict[str, float]:
    missing = [k for k in _METRICS if k not in weights]
    extra = [k for k in weights.keys() if k not in _METRICS]
    if missing:
        raise ValueError(f"weights에 누락된 키가 있음: {missing}")
    if extra:
        raise ValueError(f"weights에 불필요한 키가 있음: {extra}")

    w = {k: float(weights[k]) for k in _METRICS}
    if any(v < 0 for v in w.values()):
        raise ValueError("weights는 음수가 될 수 없음")

    s = sum(w.values())
    if require_sum_to_one:
        # 외부 산출치가 1로 딱 맞지 않을 수 있으니, 보정 or 에러 중 택1
        if not np.isclose(s, 1.0, atol=1e-6):
            raise ValueError(f"weights 합이 1이 아님: sum={s:.6f}")
    return w

# ----------------------------
# 4) 팀 점수 계산 (weights 주입형)
# ----------------------------
def team_fit_score(
    team: List[Employee],
    weights: Dict[str, float],
    dec_target: float = 0.12,
    dec_tolerance: float = 0.10,
) -> Tuple[float, Dict[str, float], Dict[str, str]]:

    w = validate_weights(weights, require_sum_to_one=True)

    trust = [m.trust for m in team]
    comm  = [m.communication for m in team]
    dec   = [m.decision_making for m in team]
    innov = [m.innovative_thinking for m in team]
    conf  = [m.conflict_management for m in team]
    role  = [m.role_clarity for m in team]
    psy   = [m.psychological_safety for m in team]

    # 팀 평균 (6개)
    trust_mean = mean(trust)
    comm_mean  = mean(comm)
    innov_mean = mean(innov)
    conf_mean  = mean(conf)
    psy_mean   = mean(psy)
    role_mean  = mean(role)

    # 의사결정은 "팀 균형" (표준편차 기반)
    dec_std = diversity(dec)
    dec_balance = balance_score(dec_std, target=dec_target, tolerance=dec_tolerance)

    driver_scores = {
        "trust": trust_mean,
        "communication": comm_mean,
        "innovative_thinking": innov_mean,
        "decision_making": dec_balance,          # ✅ 팀 균형 점수
        "conflict_management": conf_mean,
        "role_clarity": role_mean,
        "psychological_safety": psy_mean
    }

    # ✅ 가중치 적용(팀 점수)
    total = sum(w[k] * driver_scores[k] for k in _METRICS)
    total_score = float(np.clip(total * 100.0, 0, 100))

    # 설명(Notes)
    notes: Dict[str, str] = {}

    def explain_mean(name, val):
        if val >= 0.75: return f"{name} 평균이 높음(≈{val:.2f})."
        if val >= 0.55: return f"{name} 평균이 보통(≈{val:.2f})."
        return f"{name} 평균이 낮음(≈{val:.2f})."

    notes["trust"] = explain_mean("Trust", trust_mean)
    notes["communication"] = explain_mean("Communication", comm_mean)
    notes["innovative_thinking"] = explain_mean("Innovative thinking", innov_mean)
    notes["conflict_management"] = explain_mean("Conflict management", conf_mean)
    notes["role_clarity"] = explain_mean("Role clarity", role_mean)
    notes["psychological_safety"] = explain_mean("Psychological safety", psy_mean)

    if dec_balance >= 0.75:
        notes["decision_making"] = f"의사결정 다양성 적정(표준편차≈{dec_std:.2f})."
    elif dec_std < 0.06:
        notes["decision_making"] = f"의사결정이 너무 유사(표준편차≈{dec_std:.2f}) → 경직 위험."
    else:
        notes["decision_making"] = f"의사결정 다양성 큼(표준편차≈{dec_std:.2f}) → 조율 비용 증가 가능."

    return total_score, driver_scores, notes

# ----------------------------
# 5) Top-K 팀 전수 평가 (weights 주입형)
# ----------------------------
def _team_brief(drivers: Dict[str, float], notes: Dict[str, str], weights: Dict[str, float]) -> str:
    contrib = sorted(((weights[k] * drivers[k], k) for k in _METRICS), reverse=True)

    strengths = []
    for _, k in contrib:
        if k == "decision_making":
            continue
        strengths.append(k)
        if len(strengths) == 2:
            break

    weak_k = min(drivers.items(), key=lambda kv: kv[1])[0]
    s1, s2 = strengths[0], strengths[1]

    return (
        f"강점: {_DRIVER_KO[s1]}(≈{drivers[s1]:.2f}), {_DRIVER_KO[s2]}(≈{drivers[s2]:.2f}) / "
        f"리스크: {_DRIVER_KO[weak_k]}(≈{drivers[weak_k]:.2f}). {notes['decision_making']}"
    )

def topk_teams_exact(
    employees: List[Employee],
    team_size: int,
    weights: Dict[str, float],
    top_k: int = 5,
    log_every: int = 50_000,
) -> List[Dict]:

    w = validate_weights(weights, require_sum_to_one=True)

    n = len(employees)
    if team_size < 2 or team_size > n:
        raise ValueError(f"team_size={team_size} must be between 2 and len(employees)={n}")

    comb_count = math.comb(n, team_size)
    effective_top_k = min(top_k, comb_count)

    print(f"[INFO] employees={n}, team_size={team_size}, combinations={comb_count:,} (exact)")

    heap: List[Tuple[float, Tuple[int, ...]]] = []

    for c, idx in enumerate(itertools.combinations(range(n), team_size), start=1):
        team = [employees[i] for i in idx]
        score, _, _ = team_fit_score(team, weights=w)

        if len(heap) < effective_top_k:
            heapq.heappush(heap, (score, idx))
        elif score > heap[0][0]:
            heapq.heapreplace(heap, (score, idx))

        if log_every and (c % log_every == 0):
            best_now = max(heap, key=lambda x: x[0])[0] if heap else 0.0
            print(f"[PROGRESS] {c:,}/{comb_count:,} checked | best≈{best_now:.2f}")

    winners = sorted(heap, key=lambda x: x[0], reverse=True)

    results: List[Dict] = []
    for rank, (score, idx) in enumerate(winners, start=1):
        team = [employees[i] for i in idx]
        score2, drivers, notes = team_fit_score(team, weights=w)
        results.append({
            "rank": rank,
            "score": round(score2, 2),
            "members": [m.name for m in team],
            "brief": _team_brief(drivers, notes, w),
            "driver_scores_team": {k: round(v, 3) for k, v in drivers.items()},
        })
    return results

# ----------------------------
# 6) 개인 가중치 적용 (외부 weights 주입형)
# ----------------------------
def individual_weighted_breakdown(
    emp: Employee,
    weights: Dict[str, float],
) -> Tuple[Dict[str, float], Dict[str, float], float]:

    w = validate_weights(weights, require_sum_to_one=True)

    raw = {k: float(getattr(emp, k)) for k in _METRICS}
    # ✅ 가중치 적용(개인 점수)
    weighted = {k: raw[k] * w[k] for k in _METRICS}
    total_100 = float(np.clip(sum(weighted.values()) * 100.0, 0, 100))
    return raw, weighted, total_100

def print_top_teams_with_individuals(
    top_teams: List[Dict],
    employees: List[Employee],
    weights: Dict[str, float],
) -> None:

    w = validate_weights(weights, require_sum_to_one=True)

    print(f"\n=== TOP {len(top_teams)} Teams ===")
    print(f"[WEIGHTS] " + ", ".join([f"{k}={w[k]:.3f}" for k in _METRICS]))

    name_to_emp = {e.name: e for e in employees}

    for t in top_teams:
        print(f"\n#{t['rank']} | TeamScore={t['score']:.2f} | Members={', '.join(t['members'])}")
        print(f"- 요약: {t['brief']}")
        print(f"- Team driver scores: {t['driver_scores_team']}")

        print("  [Individuals | raw & weighted]")
        for member_name in t["members"]:
            emp = name_to_emp[member_name]
            raw, weighted, total_100 = individual_weighted_breakdown(emp, w)

            raw_str = ", ".join([f"{k}={raw[k]:.2f}" for k in _METRICS])
            wtd_str = ", ".join([f"{k}={weighted[k]:.3f}" for k in _METRICS])

            print(f"  - {emp.name} | indiv_total≈{total_100:.2f}")
            print(f"    raw: {raw_str}")
            print(f"    wtd: {wtd_str}")
EXTERNAL_WEIGHTS = { "innovative_thinking": 0.26, "decision_making": 0.20, "communication": 0.18, "trust": 0.16,
            "psychological_safety": 0.08, "conflict_management": 0.07, "role_clarity": 0.05 }  # 합=1
top5 = topk_teams_exact(employees=SAMPLE_EMPLOYEES_50, team_size=4, weights=EXTERNAL_WEIGHTS, top_k=5)
print_top_teams_with_individuals(top5, SAMPLE_EMPLOYEES_50, EXTERNAL_WEIGHTS)


[INFO] employees=50, team_size=4, combinations=230,300 (exact)
[PROGRESS] 50,000/230,300 checked | best≈80.32
[PROGRESS] 100,000/230,300 checked | best≈80.91
[PROGRESS] 150,000/230,300 checked | best≈80.91
[PROGRESS] 200,000/230,300 checked | best≈80.91

=== TOP 5 Teams ===
[WEIGHTS] trust=0.160, communication=0.180, decision_making=0.200, innovative_thinking=0.260, conflict_management=0.070, role_clarity=0.050, psychological_safety=0.080

#1 | TeamScore=80.91 | Members=E005, E013, E025, E032
- 요약: 강점: 혁신/아이디어(≈0.72), 커뮤니케이션(≈0.77) / 리스크: 혁신/아이디어(≈0.72). 의사결정 다양성 적정(표준편차≈0.11).
- Team driver scores: {'trust': 0.818, 'communication': 0.768, 'innovative_thinking': 0.725, 'decision_making': 0.99, 'conflict_management': 0.73, 'role_clarity': 0.765, 'psychological_safety': 0.805}
  [Individuals | raw & weighted]
  - E005 | indiv_total≈72.88
    raw: trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safe

In [12]:
# ============================
# 7) LLM 리포트 생성 (sys_prompt 붙이기)
# ============================
from openai import OpenAI

# (1) 너가 만든 sys_prompt를 여기에 넣어!
#     너무 길면, sys_prompt = """...""" 이 부분만 따로 파일에 저장해도 됨.
sys_prompt = """
[ROLE]
당신은 “AI 팀 빌딩 서비스 리포트 생성기(Consulting-grade Report Writer)”다.
출력은 반드시 한국어로 작성한다.
입력으로 제공되는 REFERENCE_MATERIALS와 USER_RUN_LOG만 사용한다. (그 외 외부 지식/추정 금지)

[INPUTS: AUTO-INJECTED]
1) REFERENCE_MATERIALS
- (A) 도메인 로직(가중치 정당화): Cynefin, Cost of Error, Task Interdependence
1. 가중치 산출의 근거가 될 도메인 프레임워크 (Domain Logic)
AI에게 이 개념들을 학습시키(프롬프트에 포함하)면 매우 논리적인 숫자를 뱉어냅니다.
A. 커네빈 프레임워크 (Cynefin Framework) - 데이브 스노든
프로젝트가 마주한 **'불확실성의 종류'**를 4단계로 나누는 이론입니다. 이를 통해 Role Clarity와 Innovation의 비중을 결정합니다.
Simple (단순형): 인과관계가 명확함 (예: 단순 제조, 데이터 입력)
👉 가중치 전략: Role Clarity ↑, Execution ↑, Innovation ↓
Complicated (난해형): 전문가가 필요한 문제 (예: 대규모 시스템 구축, 교량 건설)
👉 가중치 전략: Role Clarity (보통), Trust ↑ (전문가 간 신뢰), Communication ↑
Complex (복잡형): 해답을 모름, 실험이 필요 (예: 신약 개발, AI 모델링, 스타트업)
👉 가중치 전략: Psychological Safety ↑↑ (실패 용인), Innovative Thinking ↑, Role Clarity ↓ (유연성)
Chaotic (혼란형): 위기 상황 (예: 서버 다운, 재난 대응)
👉 가중치 전략: Decision Making ↑↑ (즉각적 행동), Trust ↑ (리더 복종)
B. 실수 비용 이론 (Cost of Error Analysis) - 배리 보햄 (Barry Boehm)
"이 프로젝트에서 실수했을 때 얼마나 치명적인가?"를 기준으로 심리적 안전감과 신뢰의 성격을 규정합니다.
High Cost (의료, 금융, 항공): 실수가 용납되지 않음.
👉 Role Clarity와 Conflict Management(교차 검증) 가중치 대폭 상승.
Low Cost (게임, 소셜앱, 프로토타이핑): 빨리 고치면 됨.
👉 Decision Making(속도)과 Innovation 가중치 상승.
C. 상호의존성 이론 (Task Interdependence) - 톰슨
팀원들이 얼마나 얽혀서 일해야 하는가?
Pooled (집합적): 각자 일해서 합침 (예: 보험 영업, 콜센터) → Role Clarity 중요.
Reciprocal (상호적): A가 B에게 주고, B가 다시 A에게 줌 (예: SW개발, 수술팀) → Communication & Trust 가중치 폭증.


- (B) McKinsey 기반 7개 지표 정의/해석(신뢰~심리적 안전)
요약
McKinsey는 팀 성과 차이를 신뢰, 커뮤니케이션, 의사결정, 혁신적 사고 같은 행동·관계 요인이 설명한다고 분석했다.
성과 유형에 따라 건강 드라이버의 상대적 중요도가 달라진다.
조직 성과 분석에서 설명 가능한 인사이트가 예측 점수만큼 중요한 요소로 작용한다.
본 프로젝트는 이러한 McKinsey의 분석을 바탕으로 Feature Definition 및 Context 기반 가중치 구조를 설정하고, 생성형 AI를 활용한 설명 모듈을 핵심 구성 요소로 두고 있다.
McKinsey 팀 건강 모델의 이론적 근거와 본 프로젝트와의 연관성
McKinsey & Company는 조직 성과를 설명하기 위해 전통적인 기술 스킬(skill)이나 역할(role) 중심의 단순 조합이 아니라, 신뢰(trust), 커뮤니케이션(communication), 의사결정(decision-making), 혁신적 사고(innovative thinking) 등과 같은 행동적·관계적 요인(health drivers) 이 조직 성과 변동에 상당한 설명력을 가진다는 분석을 제시했다. 이는 팀 매칭 모델에서 단순 유사도(similarity)만을 기반으로 팀을 구성하는 전통적 접근이 가진 한계를 명확하게 보여준다.
이러한 McKinsey의 분석은 본 프로젝트의 구조적 설계에 다음과 같은 세 가지 핵심 근거를 제공한다.
첫째, 팀 성과는 다양한 행동·관계 요인의 상호작용으로 설명된다는 McKinsey의 결론은 팀 매칭 모델에서 어떤 입력 변수(feature)가 중요한지를 정하는 기준을 제공한다. 팀 간 신뢰, 의사소통, 혁신적 사고와 같은 건강 드라이버는 단일 특성 평균이나 유사도만으로는 설명하기 어려운 팀 차이를 포착할 수 있으며, 이는 본 시스템에서 팀 적합도 점수를 산출하는 전 단계인 Feature Definition 단계의 이론적 기준이 된다.
둘째, McKinsey는 성과 유형(효율성, 성과 결과, 혁신) 별로 건강 드라이버의 상대적 중요도가 달라진다는 점을 지적하고 있다. 이는 본 프로젝트가 프로젝트 유형(task type/case context)에 따라 팀 건강 드라이버 가중치를 다르게 부여하는 설계를 정당화하는 논리적 근거를 제공한다. 즉, 동일한 팀이라도 과업 맥락에 따라 핵심 성과 드라이버가 달라질 수 있음을 시사하는 것이다.
셋째, McKinsey는 조직 성과를 개선하기 위해 설명 가능한 조직 인사이트(explainable insight)를 도출하는 것이 점수나 예측 결과만 생성하는 것 못지않게 중요하다고 분석한다. 이는 본 프로젝트에서 점수 산출만으로 끝나는 것이 아니라 생성형 AI를 활용해 점수의 이유, 리스크, 운영 가이드를 해석/설명하는 부분이 단순 점수 기반 추천 대비 실질적인 조직 의사결정 지원 면에서 더 높은 가치를 가진다는 논리적 근거로 작용한다.
따라서 본 시스템에서 McKinsey 팀 건강 모델은 단순한 “참고 문헌”을 넘어서, 머신러닝 모델 설계 이전 단계에서의 핵심 Feature 정의와 가중치 구조 설정, 그리고 설명 가능성의 중요성을 뒷받침하는 이론적 기반으로 기능한다.
 이러한 구조적 정당화는 본 프로젝트가 단순한 알고리즘 실험이 아니라 조직 행동/성과 연구의 축적된 지식을 기반으로 한 근거 기반 의사결정 지원 시스템임을 명확하게 보여준다.


- (C) 최적화/산식 코드 설명(팀 점수/개인 점수/전수탐색 로직)
import numpy as np
import math
import itertools
import heapq
from dataclasses import dataclass
from typing import List, Dict, Tuple

# ----------------------------
# 1) 데이터 구조
# ----------------------------
@dataclass(frozen=True)
class Employee:
    name: str
    trust: float
    communication: float
    decision_making: float
    innovative_thinking: float
    conflict_management: float
    role_clarity: float
    psychological_safety: float

_METRICS = [
    "trust", "communication", "decision_making",
    "innovative_thinking", "conflict_management",
    "role_clarity", "psychological_safety"
]

_DRIVER_KO = {
    "trust": "신뢰",
    "communication": "커뮤니케이션",
    "innovative_thinking": "혁신/아이디어",
    "decision_making": "의사결정 균형",
    "conflict_management": "갈등관리",
    "role_clarity": "역할명확성",
    "psychological_safety": "심리적 안전",
}


 #샘플 데이터: 50명 (E001~E050)
# ----------------------------
SAMPLE_EMPLOYEES_50: List[Employee] = [
    Employee(name="E001", trust=0.82, communication=0.78, decision_making=0.52, innovative_thinking=0.54, conflict_management=0.76, role_clarity=0.88, psychological_safety=0.73),
    Employee(name="E002", trust=0.80, communication=0.74, decision_making=0.48, innovative_thinking=0.50, conflict_management=0.72, role_clarity=0.86, psychological_safety=0.70),
    Employee(name="E003", trust=0.85, communication=0.81, decision_making=0.55, innovative_thinking=0.57, conflict_management=0.79, role_clarity=0.90, psychological_safety=0.76),
    Employee(name="E004", trust=0.78, communication=0.72, decision_making=0.44, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.84, psychological_safety=0.68),
    Employee(name="E005", trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79),
    Employee(name="E006", trust=0.76, communication=0.70, decision_making=0.46, innovative_thinking=0.47, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.66),
    Employee(name="E007", trust=0.84, communication=0.79, decision_making=0.50, innovative_thinking=0.56, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),
    Employee(name="E008", trust=0.81, communication=0.76, decision_making=0.53, innovative_thinking=0.52, conflict_management=0.75, role_clarity=0.87, psychological_safety=0.71),
    Employee(name="E009", trust=0.79, communication=0.73, decision_making=0.49, innovative_thinking=0.51, conflict_management=0.72, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E010", trust=0.86, communication=0.82, decision_making=0.56, innovative_thinking=0.58, conflict_management=0.80, role_clarity=0.91, psychological_safety=0.77),
    Employee(name="E011", trust=0.83, communication=0.77, decision_making=0.47, innovative_thinking=0.53, conflict_management=0.74, role_clarity=0.88, psychological_safety=0.72),
    Employee(name="E012", trust=0.77, communication=0.71, decision_making=0.45, innovative_thinking=0.48, conflict_management=0.70, role_clarity=0.84, psychological_safety=0.67),
    Employee(name="E013", trust=0.87, communication=0.84, decision_making=0.57, innovative_thinking=0.59, conflict_management=0.81, role_clarity=0.92, psychological_safety=0.78),
    Employee(name="E014", trust=0.82, communication=0.75, decision_making=0.51, innovative_thinking=0.55, conflict_management=0.76, role_clarity=0.87, psychological_safety=0.70),
    Employee(name="E015", trust=0.80, communication=0.74, decision_making=0.43, innovative_thinking=0.46, conflict_management=0.73, role_clarity=0.86, psychological_safety=0.68),
    Employee(name="E016", trust=0.85, communication=0.80, decision_making=0.54, innovative_thinking=0.58, conflict_management=0.78, role_clarity=0.90, psychological_safety=0.75),
    Employee(name="E017", trust=0.78, communication=0.72, decision_making=0.50, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E018", trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80),
    Employee(name="E019", trust=0.76, communication=0.70, decision_making=0.47, innovative_thinking=0.45, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.65),
    Employee(name="E020", trust=0.84, communication=0.79, decision_making=0.53, innovative_thinking=0.57, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),

    Employee(name="E021", trust=0.70, communication=0.62, decision_making=0.30, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.52, psychological_safety=0.86),
    Employee(name="E022", trust=0.66, communication=0.58, decision_making=0.78, innovative_thinking=0.90, conflict_management=0.53, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E023", trust=0.72, communication=0.70, decision_making=0.45, innovative_thinking=0.84, conflict_management=0.60, role_clarity=0.58, psychological_safety=0.80),
    Employee(name="E024", trust=0.60, communication=0.55, decision_making=0.82, innovative_thinking=0.92, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E025", trust=0.75, communication=0.68, decision_making=0.38, innovative_thinking=0.86, conflict_management=0.62, role_clarity=0.60, psychological_safety=0.83),
    Employee(name="E026", trust=0.63, communication=0.57, decision_making=0.70, innovative_thinking=0.89, conflict_management=0.54, role_clarity=0.50, psychological_safety=0.88),
    Employee(name="E027", trust=0.78, communication=0.73, decision_making=0.55, innovative_thinking=0.81, conflict_management=0.68, role_clarity=0.63, psychological_safety=0.78),
    Employee(name="E028", trust=0.65, communication=0.60, decision_making=0.25, innovative_thinking=0.90, conflict_management=0.52, role_clarity=0.47, psychological_safety=0.92),
    Employee(name="E029", trust=0.69, communication=0.64, decision_making=0.66, innovative_thinking=0.87, conflict_management=0.56, role_clarity=0.54, psychological_safety=0.85),
    Employee(name="E030", trust=0.74, communication=0.71, decision_making=0.48, innovative_thinking=0.83, conflict_management=0.61, role_clarity=0.59, psychological_safety=0.81),
    Employee(name="E031", trust=0.62, communication=0.56, decision_making=0.80, innovative_thinking=0.91, conflict_management=0.49, role_clarity=0.45, psychological_safety=0.90),
    Employee(name="E032", trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82),
    Employee(name="E033", trust=0.68, communication=0.63, decision_making=0.60, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.53, psychological_safety=0.87),
    Employee(name="E034", trust=0.61, communication=0.55, decision_making=0.74, innovative_thinking=0.92, conflict_management=0.51, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E035", trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79),
    Employee(name="E036", trust=0.64, communication=0.59, decision_making=0.83, innovative_thinking=0.90, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E037", trust=0.71, communication=0.66, decision_making=0.58, innovative_thinking=0.86, conflict_management=0.57, role_clarity=0.55, psychological_safety=0.84),
    Employee(name="E038", trust=0.67, communication=0.61, decision_making=0.28, innovative_thinking=0.89, conflict_management=0.52, role_clarity=0.49, psychological_safety=0.92),
    Employee(name="E039", trust=0.73, communication=0.69, decision_making=0.72, innovative_thinking=0.82, conflict_management=0.60, role_clarity=0.57, psychological_safety=0.80),
    Employee(name="E040", trust=0.65, communication=0.60, decision_making=0.50, innovative_thinking=0.87, conflict_management=0.54, role_clarity=0.51, psychological_safety=0.88),

    Employee(name="E041", trust=0.62, communication=0.70, decision_making=0.88, innovative_thinking=0.68, conflict_management=0.55, role_clarity=0.50, psychological_safety=0.60),
    Employee(name="E042", trust=0.58, communication=0.66, decision_making=0.92, innovative_thinking=0.72, conflict_management=0.50, role_clarity=0.48, psychological_safety=0.58),
    Employee(name="E043", trust=0.66, communication=0.73, decision_making=0.80, innovative_thinking=0.65, conflict_management=0.60, role_clarity=0.52, psychological_safety=0.62),
    Employee(name="E044", trust=0.55, communication=0.61, decision_making=0.95, innovative_thinking=0.75, conflict_management=0.48, role_clarity=0.45, psychological_safety=0.55),
    Employee(name="E045", trust=0.70, communication=0.76, decision_making=0.78, innovative_thinking=0.63, conflict_management=0.62, role_clarity=0.55, psychological_safety=0.65),
    Employee(name="E046", trust=0.60, communication=0.68, decision_making=0.85, innovative_thinking=0.70, conflict_management=0.53, role_clarity=0.49, psychological_safety=0.59),
    Employee(name="E047", trust=0.64, communication=0.72, decision_making=0.90, innovative_thinking=0.74, conflict_management=0.56, role_clarity=0.51, psychological_safety=0.61),
    Employee(name="E048", trust=0.57, communication=0.63, decision_making=0.82, innovative_thinking=0.67, conflict_management=0.49, role_clarity=0.46, psychological_safety=0.56),
    Employee(name="E049", trust=0.71, communication=0.78, decision_making=0.76, innovative_thinking=0.62, conflict_management=0.63, role_clarity=0.56, psychological_safety=0.67),
    Employee(name="E050", trust=0.59, communication=0.67, decision_making=0.93, innovative_thinking=0.73, conflict_management=0.52, role_clarity=0.48, psychological_safety=0.58),
]

# ----------------------------
# 2) 기본 함수
# ----------------------------
def mean(vals) -> float:
    return float(np.mean(vals))

def diversity(vals) -> float:
    return float(np.std(vals))

def balance_score(std: float, target: float, tolerance: float) -> float:
    return float(np.exp(-((std - target) ** 2) / (2 * (tolerance ** 2))))

# ----------------------------
# 3) weights 검증 (외부 주입용 안전장치)
# ----------------------------
def validate_weights(weights: Dict[str, float], require_sum_to_one: bool = True) -> Dict[str, float]:
    missing = [k for k in _METRICS if k not in weights]
    extra = [k for k in weights.keys() if k not in _METRICS]
    if missing:
        raise ValueError(f"weights에 누락된 키가 있음: {missing}")
    if extra:
        raise ValueError(f"weights에 불필요한 키가 있음: {extra}")

    w = {k: float(weights[k]) for k in _METRICS}
    if any(v < 0 for v in w.values()):
        raise ValueError("weights는 음수가 될 수 없음")

    s = sum(w.values())
    if require_sum_to_one:
        # 외부 산출치가 1로 딱 맞지 않을 수 있으니, 보정 or 에러 중 택1
        if not np.isclose(s, 1.0, atol=1e-6):
            raise ValueError(f"weights 합이 1이 아님: sum={s:.6f}")
    return w

# ----------------------------
# 4) 팀 점수 계산 (weights 주입형)
# ----------------------------
def team_fit_score(
    team: List[Employee],
    weights: Dict[str, float],
    dec_target: float = 0.12,
    dec_tolerance: float = 0.10,
) -> Tuple[float, Dict[str, float], Dict[str, str]]:

    w = validate_weights(weights, require_sum_to_one=True)

    trust = [m.trust for m in team]
    comm  = [m.communication for m in team]
    dec   = [m.decision_making for m in team]
    innov = [m.innovative_thinking for m in team]
    conf  = [m.conflict_management for m in team]
    role  = [m.role_clarity for m in team]
    psy   = [m.psychological_safety for m in team]

    # 팀 평균 (6개)
    trust_mean = mean(trust)
    comm_mean  = mean(comm)
    innov_mean = mean(innov)
    conf_mean  = mean(conf)
    psy_mean   = mean(psy)
    role_mean  = mean(role)

    # 의사결정은 "팀 균형" (표준편차 기반)
    dec_std = diversity(dec)
    dec_balance = balance_score(dec_std, target=dec_target, tolerance=dec_tolerance)

    driver_scores = {
        "trust": trust_mean,
        "communication": comm_mean,
        "innovative_thinking": innov_mean,
        "decision_making": dec_balance,          # ✅ 팀 균형 점수
        "conflict_management": conf_mean,
        "role_clarity": role_mean,
        "psychological_safety": psy_mean
    }

    # ✅ 가중치 적용(팀 점수)
    total = sum(w[k] * driver_scores[k] for k in _METRICS)
    total_score = float(np.clip(total * 100.0, 0, 100))

    # 설명(Notes)
    notes: Dict[str, str] = {}

    def explain_mean(name, val):
        if val >= 0.75: return f"{name} 평균이 높음(≈{val:.2f})."
        if val >= 0.55: return f"{name} 평균이 보통(≈{val:.2f})."
        return f"{name} 평균이 낮음(≈{val:.2f})."

    notes["trust"] = explain_mean("Trust", trust_mean)
    notes["communication"] = explain_mean("Communication", comm_mean)
    notes["innovative_thinking"] = explain_mean("Innovative thinking", innov_mean)
    notes["conflict_management"] = explain_mean("Conflict management", conf_mean)
    notes["role_clarity"] = explain_mean("Role clarity", role_mean)
    notes["psychological_safety"] = explain_mean("Psychological safety", psy_mean)

    if dec_balance >= 0.75:
        notes["decision_making"] = f"의사결정 다양성 적정(표준편차≈{dec_std:.2f})."
    elif dec_std < 0.06:
        notes["decision_making"] = f"의사결정이 너무 유사(표준편차≈{dec_std:.2f}) → 경직 위험."
    else:
        notes["decision_making"] = f"의사결정 다양성 큼(표준편차≈{dec_std:.2f}) → 조율 비용 증가 가능."

    return total_score, driver_scores, notes

# ----------------------------
# 5) Top-K 팀 전수 평가 (weights 주입형)
# ----------------------------
def _team_brief(drivers: Dict[str, float], notes: Dict[str, str], weights: Dict[str, float]) -> str:
    contrib = sorted(((weights[k] * drivers[k], k) for k in _METRICS), reverse=True)

    strengths = []
    for _, k in contrib:
        if k == "decision_making":
            continue
        strengths.append(k)
        if len(strengths) == 2:
            break

    weak_k = min(drivers.items(), key=lambda kv: kv[1])[0]
    s1, s2 = strengths[0], strengths[1]

    return (
        f"강점: {_DRIVER_KO[s1]}(≈{drivers[s1]:.2f}), {_DRIVER_KO[s2]}(≈{drivers[s2]:.2f}) / "
        f"리스크: {_DRIVER_KO[weak_k]}(≈{drivers[weak_k]:.2f}). {notes['decision_making']}"
    )

def topk_teams_exact(
    employees: List[Employee],
    team_size: int,
    weights: Dict[str, float],
    top_k: int = 5,
    log_every: int = 50_000,
) -> List[Dict]:

    w = validate_weights(weights, require_sum_to_one=True)

    n = len(employees)
    if team_size < 2 or team_size > n:
        raise ValueError(f"team_size={team_size} must be between 2 and len(employees)={n}")

    comb_count = math.comb(n, team_size)
    effective_top_k = min(top_k, comb_count)

    print(f"[INFO] employees={n}, team_size={team_size}, combinations={comb_count:,} (exact)")

    heap: List[Tuple[float, Tuple[int, ...]]] = []

    for c, idx in enumerate(itertools.combinations(range(n), team_size), start=1):
        team = [employees[i] for i in idx]
        score, _, _ = team_fit_score(team, weights=w)

        if len(heap) < effective_top_k:
            heapq.heappush(heap, (score, idx))
        elif score > heap[0][0]:
            heapq.heapreplace(heap, (score, idx))

        if log_every and (c % log_every == 0):
            best_now = max(heap, key=lambda x: x[0])[0] if heap else 0.0
            print(f"[PROGRESS] {c:,}/{comb_count:,} checked | best≈{best_now:.2f}")

    winners = sorted(heap, key=lambda x: x[0], reverse=True)

    results: List[Dict] = []
    for rank, (score, idx) in enumerate(winners, start=1):
        team = [employees[i] for i in idx]
        score2, drivers, notes = team_fit_score(team, weights=w)
        results.append({
            "rank": rank,
            "score": round(score2, 2),
            "members": [m.name for m in team],
            "brief": _team_brief(drivers, notes, w),
            "driver_scores_team": {k: round(v, 3) for k, v in drivers.items()},
        })
    return results

# ----------------------------
# 6) 개인 가중치 적용 (외부 weights 주입형)
# ----------------------------
def individual_weighted_breakdown(
    emp: Employee,
    weights: Dict[str, float],
) -> Tuple[Dict[str, float], Dict[str, float], float]:

    w = validate_weights(weights, require_sum_to_one=True)

    raw = {k: float(getattr(emp, k)) for k in _METRICS}
    # ✅ 가중치 적용(개인 점수)
    weighted = {k: raw[k] * w[k] for k in _METRICS}
    total_100 = float(np.clip(sum(weighted.values()) * 100.0, 0, 100))
    return raw, weighted, total_100

def print_top_teams_with_individuals(
    top_teams: List[Dict],
    employees: List[Employee],
    weights: Dict[str, float],
) -> None:

    w = validate_weights(weights, require_sum_to_one=True)

    print(f"\n=== TOP {len(top_teams)} Teams ===")
    print(f"[WEIGHTS] " + ", ".join([f"{k}={w[k]:.3f}" for k in _METRICS]))

    name_to_emp = {e.name: e for e in employees}

    for t in top_teams:
        print(f"\n#{t['rank']} | TeamScore={t['score']:.2f} | Members={', '.join(t['members'])}")
        print(f"- 요약: {t['brief']}")
        print(f"- Team driver scores: {t['driver_scores_team']}")

        print("  [Individuals | raw & weighted]")
        for member_name in t["members"]:
            emp = name_to_emp[member_name]
            raw, weighted, total_100 = individual_weighted_breakdown(emp, w)

            raw_str = ", ".join([f"{k}={raw[k]:.2f}" for k in _METRICS])
            wtd_str = ", ".join([f"{k}={weighted[k]:.3f}" for k in _METRICS])

            print(f"  - {emp.name} | indiv_total≈{total_100:.2f}")
            print(f"    raw: {raw_str}")
            print(f"    wtd: {wtd_str}")
EXTERNAL_WEIGHTS = { "innovative_thinking": 0.26, "decision_making": 0.20, "communication": 0.18, "trust": 0.16,
            "psychological_safety": 0.08, "conflict_management": 0.07, "role_clarity": 0.05 }  # 합=1
top5 = topk_teams_exact(employees=SAMPLE_EMPLOYEES_50, team_size=4, weights=EXTERNAL_WEIGHTS, top_k=5)
print_top_teams_with_individuals(top5, SAMPLE_EMPLOYEES_50, EXTERNAL_WEIGHTS)

2) USER_RUN_LOG
- 콘솔 로그(최소 포함): employees, team_size, combinations(exact), WEIGHTS, TOP5 팀(랭킹/점수/멤버/팀 driver scores), 개인 raw/weighted/indiv_total
[INFO] employees=50, team_size=4, combinations=230,300 (exact)
[PROGRESS] 50,000/230,300 checked | best≈80.32
[PROGRESS] 100,000/230,300 checked | best≈80.91
[PROGRESS] 150,000/230,300 checked | best≈80.91
[PROGRESS] 200,000/230,300 checked | best≈80.91

=== TOP 5 Teams ===
[WEIGHTS] trust=0.160, communication=0.180, decision_making=0.200, innovative_thinking=0.260, conflict_management=0.070, role_clarity=0.050, psychological_safety=0.080

#1 | TeamScore=80.91 | Members=E005, E013, E025, E032
- 요약: 강점: 혁신/아이디어(≈0.72), 커뮤니케이션(≈0.77) / 리스크: 혁신/아이디어(≈0.72). 의사결정 다양성 적정(표준편차≈0.11).
- Team driver scores: {'trust': 0.818, 'communication': 0.768, 'innovative_thinking': 0.725, 'decision_making': 0.99, 'conflict_management': 0.73, 'role_clarity': 0.765, 'psychological_safety': 0.805}
  [Individuals | raw & weighted]
  - E005 | indiv_total≈72.88
    raw: trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79
    wtd: trust=0.141, communication=0.149, decision_making=0.116, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.046, psychological_safety=0.063
  - E013 | indiv_total≈72.29
    raw: trust=0.87, communication=0.84, decision_making=0.57, innovative_thinking=0.59, conflict_management=0.81, role_clarity=0.92, psychological_safety=0.78
    wtd: trust=0.139, communication=0.151, decision_making=0.114, innovative_thinking=0.153, conflict_management=0.057, role_clarity=0.046, psychological_safety=0.062
  - E025 | indiv_total≈68.18
    raw: trust=0.75, communication=0.68, decision_making=0.38, innovative_thinking=0.86, conflict_management=0.62, role_clarity=0.60, psychological_safety=0.83
    wtd: trust=0.120, communication=0.122, decision_making=0.076, innovative_thinking=0.224, conflict_management=0.043, role_clarity=0.030, psychological_safety=0.066
  - E032 | indiv_total≈68.73
    raw: trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82
    wtd: trust=0.123, communication=0.130, decision_making=0.070, innovative_thinking=0.221, conflict_management=0.047, role_clarity=0.031, psychological_safety=0.066

#2 | TeamScore=80.90 | Members=E005, E018, E025, E032
- 요약: 강점: 혁신/아이디어(≈0.73), 커뮤니케이션(≈0.76) / 리스크: 혁신/아이디어(≈0.73). 의사결정 다양성 적정(표준편차≈0.10).
- Team driver scores: {'trust': 0.82, 'communication': 0.765, 'innovative_thinking': 0.728, 'decision_making': 0.982, 'conflict_management': 0.732, 'role_clarity': 0.768, 'psychological_safety': 0.81}
  [Individuals | raw & weighted]
  - E005 | indiv_total≈72.88
    raw: trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79
    wtd: trust=0.141, communication=0.149, decision_making=0.116, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.046, psychological_safety=0.063
  - E018 | indiv_total≈72.41
    raw: trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80
    wtd: trust=0.141, communication=0.149, decision_making=0.110, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.047, psychological_safety=0.064
  - E025 | indiv_total≈68.18
    raw: trust=0.75, communication=0.68, decision_making=0.38, innovative_thinking=0.86, conflict_management=0.62, role_clarity=0.60, psychological_safety=0.83
    wtd: trust=0.120, communication=0.122, decision_making=0.076, innovative_thinking=0.224, conflict_management=0.043, role_clarity=0.030, psychological_safety=0.066
  - E032 | indiv_total≈68.73
    raw: trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82
    wtd: trust=0.123, communication=0.130, decision_making=0.070, innovative_thinking=0.221, conflict_management=0.047, role_clarity=0.031, psychological_safety=0.066

#3 | TeamScore=80.87 | Members=E005, E013, E032, E035
- 요약: 강점: 혁신/아이디어(≈0.72), 커뮤니케이션(≈0.78) / 리스크: 혁신/아이디어(≈0.72). 의사결정 다양성 적정(표준편차≈0.10).
- Team driver scores: {'trust': 0.82, 'communication': 0.782, 'innovative_thinking': 0.72, 'decision_making': 0.977, 'conflict_management': 0.74, 'role_clarity': 0.775, 'psychological_safety': 0.795}
  [Individuals | raw & weighted]
  - E005 | indiv_total≈72.88
    raw: trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79
    wtd: trust=0.141, communication=0.149, decision_making=0.116, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.046, psychological_safety=0.063
  - E013 | indiv_total≈72.29
    raw: trust=0.87, communication=0.84, decision_making=0.57, innovative_thinking=0.59, conflict_management=0.81, role_clarity=0.92, psychological_safety=0.78
    wtd: trust=0.139, communication=0.151, decision_making=0.114, innovative_thinking=0.153, conflict_management=0.057, role_clarity=0.046, psychological_safety=0.062
  - E032 | indiv_total≈68.73
    raw: trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82
    wtd: trust=0.123, communication=0.130, decision_making=0.070, innovative_thinking=0.221, conflict_management=0.047, role_clarity=0.031, psychological_safety=0.066
  - E035 | indiv_total≈69.86
    raw: trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79
    wtd: trust=0.122, communication=0.133, decision_making=0.084, innovative_thinking=0.218, conflict_management=0.046, role_clarity=0.032, psychological_safety=0.063

#4 | TeamScore=80.80 | Members=E005, E018, E032, E035
- 요약: 강점: 혁신/아이디어(≈0.72), 커뮤니케이션(≈0.78) / 리스크: 혁신/아이디어(≈0.72). 의사결정 다양성 적정(표준편차≈0.09).
- Team driver scores: {'trust': 0.823, 'communication': 0.78, 'innovative_thinking': 0.722, 'decision_making': 0.967, 'conflict_management': 0.743, 'role_clarity': 0.778, 'psychological_safety': 0.8}
  [Individuals | raw & weighted]
  - E005 | indiv_total≈72.88
    raw: trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79
    wtd: trust=0.141, communication=0.149, decision_making=0.116, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.046, psychological_safety=0.063
  - E018 | indiv_total≈72.41
    raw: trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80
    wtd: trust=0.141, communication=0.149, decision_making=0.110, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.047, psychological_safety=0.064
  - E032 | indiv_total≈68.73
    raw: trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82
    wtd: trust=0.123, communication=0.130, decision_making=0.070, innovative_thinking=0.221, conflict_management=0.047, role_clarity=0.031, psychological_safety=0.066
  - E035 | indiv_total≈69.86
    raw: trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79
    wtd: trust=0.122, communication=0.133, decision_making=0.084, innovative_thinking=0.218, conflict_management=0.046, role_clarity=0.032, psychological_safety=0.063

#5 | TeamScore=80.80 | Members=E018, E029, E032, E035
- 요약: 강점: 혁신/아이디어(≈0.79), 커뮤니케이션(≈0.73) / 리스크: 갈등관리(≈0.68). 의사결정 다양성 적정(표준편차≈0.12).
- Team driver scores: {'trust': 0.775, 'communication': 0.732, 'innovative_thinking': 0.79, 'decision_making': 1.0, 'conflict_management': 0.677, 'role_clarity': 0.683, 'psychological_safety': 0.815}
  [Individuals | raw & weighted]
  - E018 | indiv_total≈72.41
    raw: trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80
    wtd: trust=0.141, communication=0.149, decision_making=0.110, innovative_thinking=0.156, conflict_management=0.057, role_clarity=0.047, psychological_safety=0.064
  - E029 | indiv_total≈71.80
    raw: trust=0.69, communication=0.64, decision_making=0.66, innovative_thinking=0.87, conflict_management=0.56, role_clarity=0.54, psychological_safety=0.85
    wtd: trust=0.110, communication=0.115, decision_making=0.132, innovative_thinking=0.226, conflict_management=0.039, role_clarity=0.027, psychological_safety=0.068
  - E032 | indiv_total≈68.73
    raw: trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82
    wtd: trust=0.123, communication=0.130, decision_making=0.070, innovative_thinking=0.221, conflict_management=0.047, role_clarity=0.031, psychological_safety=0.066
  - E035 | indiv_total≈69.86
    raw: trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79
    wtd: trust=0.122, communication=0.133, decision_making=0.084, innovative_thinking=0.218, conflict_management=0.046, role_clarity=0.032, psychological_safety=0.063


3) (OPTIONAL) USER_REQUIREMENTS
- 없을 수도 있음. 있다면 다음만 포함:
  - 프로젝트 맥락: Cynefin 유형, Cost of Error(High/Low), Interdependence(Pooled/Reciprocal)
  - 리포트 톤: 경영진용/실무용
  - 강조점: “가중치 정당성” vs “산식/전수탐색 투명성” (기본은 균형)

[NON-NEGOTIABLE GROUNDING RULES]
- 숫자(가중치/점수/팀/개인 값)는 USER_RUN_LOG에서만 가져와 표기한다. 재계산은 “검증” 목적만 허용하며, 표기값은 로그 우선.
- 지표 의미/이론/정당화/방법론 설명은 REFERENCE_MATERIALS에서만 요약한다.
- 로그/문서에 없는 사실(개인 성향·직무·조직 상황)을 단정하지 않는다. 필요한 경우 “가정:” 라벨로만 제시한다.
- 출력은 아래 OUTPUT CONTRACT를 100% 준수한다.

[STEP 1 — PARSE USER_RUN_LOG (구조화)]
다음 항목을 반드시 추출해 구조화하라:
- meta: employees, team_size, combinations(exact), progress(있으면), weights(7개)
- teams[1..5]: rank, TeamScore, Members(4명), team_driver_scores(7개), brief(있으면)
- individuals: 각 팀의 4명에 대해 raw(7개), weighted(7개), indiv_total
누락 항목이 있으면 “missing_fields”에 기록하고, 리포트의 ‘모델 범위/제약’ 섹션에 반영하라.

[STEP 2 — COMPUTE EXPLANATION FEATURES (설득용 파생값)]
(표기값은 로그를 그대로 쓰되) 아래 파생값을 계산해 ‘설명’에만 사용하라:
- team_contrib[k] = weight[k] * team_driver_score[k]  (k=7개)
- team_top_strengths: contrib 상위 2개(단, decision_making은 별도 표기)
- team_main_risk: driver_score 최저 1개 + (필요시 decision_making note)
- individual_contrib[k] = weight[k] * raw[k]
- individual_high/low: raw 기준 상·하위 1~2개 + 가중치 반영(개인 contrib)

[STEP 3 — METHODOLOGY (투명성: 산식+전수탐색) 작성]
다음 내용을 “짧지만 완결되게” 반드시 서술하라:
- 7개 지표 정의(REFERENCE_MATERIALS 기반 요약)
- 팀 점수 산식:
  - 6개 지표는 팀 평균(mean)
  - decision_making은 팀 내 표준편차 기반 ‘균형 점수’(dec_balance)임을 명확히
  - 최종 TeamScore = 100 * Σ(weight * driver_score), 0~100
- 개인 점수 산식:
  - weighted = raw * weight
  - indiv_total = 100 * Σ(weighted)
- 탐색 방법:
  - combinations = C(employees, team_size) (exact) 값을 그대로 제시
  - 전수탐색으로 모든 조합 평가 후 Top-5 선별(Top-k 유지 방식 요약)

[STEP 4 — WEIGHT JUSTIFICATION (도메인 로직으로 설득) 작성]
- 아래 3개 매핑 표를 반드시 작성(REFERENCE_MATERIALS 기반):
  1) Cynefin(단순/난해/복잡/혼란) → 가중치 전략(어떤 지표가 왜 ↑/↓)
  2) Cost of Error(High/Low) → 가중치 전략
  3) Interdependence(Pooled/Reciprocal) → 가중치 전략
- USER_REQUIREMENTS가 있으면: 그 맥락으로 “현재 WEIGHTS가 왜 합리적인지”를 5~7줄로 연결.
- USER_REQUIREMENTS가 없으면:
  - “가정:”으로 현재 WEIGHTS 패턴을 해석해 가능 맥락 2가지를 제시하고,
  - 각 맥락이 의미하는 가중치 방향성과 현재 WEIGHTS가 맞는지/어긋나는지 논리적으로 설명.

[OUTPUT CONTRACT — 반드시 2개 산출물]
출력은 반드시 아래 순서로 2개를 모두 제공한다.

(1) MARKDOWN_REPORT (사람이 읽는 리포트)
- 길이: ‘짧게’가 아니라 ‘리포트답게’(요약+근거+권고 포함). 단, 불필요한 미사여구 금지.
- 고정 섹션/순서(헤더 제목 동일하게 유지):
  1. Executive Summary
     - Top-1 추천 결론(팀명/점수/핵심 근거 3개)
     - Top-5 전체 비교에서 드러나는 패턴 2개(예: 혁신 강세, 의사결정 균형 등)
  2. Inputs & Scope
     - 입력(REFERENCE_MATERIALS/USER_RUN_LOG)과 자동주입 가정
     - 모델 범위/제약(누락 데이터, 측정 범위 등) — “missing_fields” 기반으로 작성
  3. Methodology (Score Transparency)
     - 팀 점수/개인 점수/전수탐색을 짧고 명확하게
  4. Weight Rationale (Domain Logic)
     - 3개 매핑 표 + 현재 WEIGHTS 정당화(또는 가정 시나리오)
  5. Top-5 Overview Dashboard
     - 팀 A~E 카드형 요약(각 카드: TeamScore, 멤버, 강점2, 리스크1)
     - 7개 지표의 팀별 비교표(숫자 + 한줄 해석)
  6. Team Detail — Team A (Rank #1)
     - Team Snapshot: 멤버/팀 점수/WEIGHTS 요약
     - Team Radar 해석(7축)
     - Why this team ranks #1: contrib 기반 근거(가중치×driver 상위 2개 + decision_making 별도)
     - Risks & Warnings: 최저 driver + decision_making 균형 문장
     - Recommended Actions: 3~5개(리스크와 직접 연결)
     - Individuals (4명): 각 개인별 “요약 3줄 + 표 + 레이더(데이터 스펙 참조)” 
  7. Team Detail — Team B (Rank #2)
  8. Team Detail — Team C (Rank #3)
  9. Team Detail — Team D (Rank #4)
  10. Team Detail — Team E (Rank #5)
     - Team B~E는 A와 동일 구조로 작성하되, 불필요한 반복 문장은 줄이고 “차별점(왜 A보다 낮은지)”을 반드시 2줄 포함.
  11. Appendix
     - 용어(7개 지표, decision_making ‘균형 점수’ 정의)
     - 로그 핵심 발췌(짧게)

(2) UI_SPEC_JSON (렌더링용 — 차트/표 데이터)
- JSON은 파싱 가능해야 하며, 키 이름 고정.
- 반드시 포함:
  - meta: employees, team_size, combinations_exact, weights(7개)
  - axes_order 고정:
    ["trust","communication","decision_making","innovative_thinking","conflict_management","role_clarity","psychological_safety"]
  - teams[5]:
    - team_id("A"~"E"), rank, team_score_100, members[4]
    - team_driver_scores_0to1 (7개)
    - team_driver_scores_0to5 (7개: *5 변환)
    - team_contrib (7개: weight*driver)
    - team_radar_spec: {type:"radar", axes_ko, values_0to5, min:0, max:5}
    - individuals[4] 각각:
      - member_id, indiv_total_100
      - raw_0to1(7개), raw_0to5(7개)
      - weighted(7개)
      - individual_contrib(7개: weight*raw)
      - individual_radar_spec: {type:"radar", axes_ko, values_0to5, min:0, max:5}
  - overview_compare:
    - table_teams_x_drivers: 팀×7지표 매트릭스(0~5)
    - notes: 각 팀의 강점2/리스크1 텍스트(로그 기반)

[STYLE RULES]
- 리포트 문장은 “근거 → 해석 → 의미(의사결정)” 순서로 작성한다.
- 팀 간 비교 문장은 반드시 ‘차이’(예: A vs B)로 작성한다. (단순 나열 금지)
- “왜”를 말할 때는 반드시 contrib(가중치×점수) 또는 raw/weighted를 연결해 설명한다.
- 같은 문장 패턴을 반복하지 않는다(문장 변주).
- 과장/감성/홍보 문구 금지. 숫자와 논리로 설득한다.

[FAIL-SAFE]
- USER_RUN_LOG에 특정 팀/개인 항목이 누락되면:
  - 해당 항목은 UI_SPEC_JSON에 null로 넣고,
  - MARKDOWN_REPORT의 ‘Inputs & Scope’에 누락 사실과 영향만 명확히 기록한다."""

# (2) top5 결과를 USER_RUN_LOG 형태로 변환해주는 함수
def build_user_run_log_text(top_teams: List[Dict], employees: List[Employee], weights: Dict[str, float]) -> str:
    w = validate_weights(weights, require_sum_to_one=True)
    name_to_emp = {e.name: e for e in employees}

    lines = []
    lines.append(f"[INFO] employees={len(employees)}, team_size=4, combinations={math.comb(len(employees), 4):,} (exact)")
    lines.append("")
    lines.append(f"=== TOP {len(top_teams)} Teams ===")
    lines.append("[WEIGHTS] " + ", ".join([f"{k}={w[k]:.3f}" for k in _METRICS]))
    lines.append("")

    for t in top_teams:
        lines.append(f"#{t['rank']} | TeamScore={t['score']:.2f} | Members={', '.join(t['members'])}")
        lines.append(f"- 요약: {t.get('brief','')}")
        lines.append(f"- Team driver scores: {t.get('driver_scores_team',{})}")
        lines.append("  [Individuals | raw & weighted]")

        for member_name in t["members"]:
            emp = name_to_emp[member_name]
            raw, weighted, total_100 = individual_weighted_breakdown(emp, w)

            raw_str = ", ".join([f"{k}={raw[k]:.2f}" for k in _METRICS])
            wtd_str = ", ".join([f"{k}={weighted[k]:.3f}" for k in _METRICS])

            # 여기서 '≈' 대신 '~' 사용
            lines.append(f"  - {emp.name} | indiv_total~{total_100:.2f}")
            lines.append(f"    raw: {raw_str}")
            lines.append(f"    wtd: {wtd_str}")

        lines.append("")

    return "\n".join(lines)

# (3) sys_prompt 안에 USER_RUN_LOG를 자동으로 끼워넣기
#     너 sys_prompt에 "2) USER_RUN_LOG" 섹션이 있으면,
#     그 자리를 통째로 덮어쓰는 방식은 케이스가 갈려서…
#     가장 쉬운 방법은 sys_prompt 맨 끝에 USER_RUN_LOG를 추가하는 거야.
user_run_log = build_user_run_log_text(top5, SAMPLE_EMPLOYEES_50, EXTERNAL_WEIGHTS)

final_system_prompt = sys_prompt + "\n\n[USER_RUN_LOG]\n" + user_run_log

# (4) 모델 호출
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": final_system_prompt},
        {"role": "user", "content": "OUTPUT CONTRACT에 따라 (1) MARKDOWN_REPORT와 (2) UI_SPEC_JSON을 순서대로 출력해줘. 반드시 둘 다 제공해."}
    ],
)

print(response.output_text)


# (1) MARKDOWN_REPORT

## Executive Summary
- **Top-1 추천 결론:** 
  - 팀명: **#1**
  - 점수: **80.91**
  - 핵심 근거: 
    1. 혁신적 사고(≈0.72) 강점.
    2. 강한 커뮤니케이션(≈0.77).
    3. 의사결정 다양성 적정(표준편차≈0.11).

- **Top-5 전체 비교에서 드러나는 패턴:**
  1. 모든 팀은 혁신적 사고에서 강점을 보이며, 이는 팀 성과에 긍정적 영향을 미친다.
  2. 의사결정의 균형 점수는 모든 팀에서 적정 수준으로 나타났고, 이는 조율 비용을 줄이고 팀워크를 강화하는 요소가 된다.

## Inputs & Scope
- **입력:**
  - REFERENCE_MATERIALS: Cynefin, Cost of Error, Task Interdependence 이론을 바탕으로 한 팀 매칭 모델.
  - USER_RUN_LOG: 팀의 구성, 점수, 개인별 평가.
  
- **모델 범위/제약:**
  - 누락 데이터: Team A~E의 개인별 세부 특성 정보. 
  - 이는 각 개인의 특성을 완전히 반영하지 못해, 전반적인 팀 분석에 제약을 줄 수 있다.

## Methodology (Score Transparency)
- **팀 점수 산식:**
  - 팀 평균(6개 지표) 기반으로 각 지표 점수를 산출.
  - 의사결정은 팀 내 표준편차를 바탕으로 한 ‘균형 점수(dec_balance)’ 사용.
  - 최종 TeamScore = 100 * Σ(weight * driver_score), 범위 0~100.

- **개인 점수 산식:**
  - weighted = raw * weight
  - indiv_total = 100 * Σ(weighted)

- **탐색 방법:**
  - 조합 수: C(50, 4) = 230,300.
  - 모든 조합을 전수탐색 후 Top-5 선별.

## Weight Rationale (Domain Logic)

1. *